In [ ]:
pip install tensorflow


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive


In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
folder_path = '/content/drive/My Drive/data'
import os
contents = os.listdir(folder_path)
print(contents)


['validation', 'training']


In [ ]:
pip install numpy opencv-python


In [ ]:
import cv2
import numpy as np

def load_and_preprocess_image(path, size=(224, 224)):
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, size, interpolation=cv2.INTER_AREA)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image / 255.0
    return image

from tensorflow.keras.preprocessing.image import ImageDataGenerator

def create_augmented_data_generator():
    # Initialize the ImageDataGenerator with augmentation parameters
    train_datagen = ImageDataGenerator(
        rescale=1./255,   # Normalize images
        rotation_range=40,  # Random rotations between 0-40 degrees
        width_shift_range=0.2,  # Random horizontal shifts
        height_shift_range=0.2,  # Random vertical shifts
        shear_range=0.2,  # Shearing
        zoom_range=0.2,  # Zooming
        horizontal_flip=True,  # Horizontal flipping
        fill_mode='nearest'  # Strategy to fill newly created pixels
    )
    return train_datagen


train_datagen = create_augmented_data_generator()
train_generator = train_datagen.flow_from_directory(
    directory='/content/drive/My Drive/data/training',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)




Found 79 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def create_model(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])


    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = create_model((224, 224, 3))


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    '/content/drive/My Drive/data/training',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    '/content/drive/My Drive/data/validation',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)


Found 79 images belonging to 2 classes.
Found 10 images belonging to 2 classes.


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=3,  # Since you have 79 training images and a batch size of 32, set steps_per_epoch to (total_images / batch_size)
    epochs=10,
    validation_data=validation_generator,
    validation_steps=1
)
model.save('/content/drive/My Drive/model.h5')

Epoch 1/10
3/3 [==============================] - 13s 3s/step - loss: 0.6547 - accuracy: 0.6329 - val_loss: 0.7560 - val_accuracy: 0.5000
Epoch 2/10
3/3 [==============================] - 13s 3s/step - loss: 0.6784 - accuracy: 0.6329 - val_loss: 0.7425 - val_accuracy: 0.5000
Epoch 3/10
3/3 [==============================] - 13s 5s/step - loss: 0.6540 - accuracy: 0.6329 - val_loss: 0.7346 - val_accuracy: 0.5000
Epoch 4/10
3/3 [==============================] - 14s 3s/step - loss: 0.6526 - accuracy: 0.6329 - val_loss: 0.7276 - val_accuracy: 0.5000
Epoch 5/10
3/3 [==============================] - 13s 4s/step - loss: 0.6640 - accuracy: 0.6329 - val_loss: 0.7062 - val_accuracy: 0.5000
Epoch 6/10
3/3 [==============================] - 12s 3s/step - loss: 0.6718 - accuracy: 0.6329 - val_loss: 0.7176 - val_accuracy: 0.5000
Epoch 7/10
3/3 [==============================] - 13s 5s/step - loss: 0.6683 - accuracy: 0.6329 - val_loss: 0.7113 - val_accuracy: 0.5000
Epoch 8/10
3/3 [==================

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
train_loss, train_accuracy = model.evaluate(train_generator)
print(f"Training Accuracy: {train_accuracy}")
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_accuracy}")


3/3 [==============================] - 5s 951ms/step - loss: 0.6573 - accuracy: 0.6329
Training Accuracy: 0.6329113841056824
1/1 [==============================] - 0s 463ms/step - loss: 0.7277 - accuracy: 0.5000
Validation Accuracy: 0.5


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import cv2

def load_and_prepare_image(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    return img_array

model = tf.keras.models.load_model('/content/drive/My Drive/model.h5')

new_image_path = '/content/WIN_20240429_15_05_56_Pro.jpg'
new_image = load_and_prepare_image(new_image_path)

predictions = model.predict(new_image)
predicted_class = 'Good Posture' if predictions[0] > 0.5 else 'Bad Posture'

print(f"The image was classified as: {predicted_class}")


1/1 [==============================] - 0s 164ms/step
The image was classified as: Bad Posture


# TRIAL

```
# This is formatted as code
```



In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Step 1: Define the CNN model
def create_model(input_shape):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification
    ])
    return model

# Step 2: Load and preprocess the data
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/My Drive/data/training',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    '/content/drive/My Drive/data/validation',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Step 3: Create and compile the model
model = create_model((224, 224, 3))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 4: Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=3,
    epochs=20,  # Increase epochs to 20
    validation_data=validation_generator,
    validation_steps=1
)

# Step 5: Save the model
model.save('/content/drive/My Drive/model_try1.h5')

# Step 6: Evaluate the model
train_loss, train_accuracy = model.evaluate(train_generator)
print(f"Training Accuracy: {train_accuracy}")

val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_accuracy}")


Found 79 images belonging to 2 classes.
Found 10 images belonging to 2 classes.
Epoch 1/20
3/3 [==============================] - 15s 4s/step - loss: 681.4786 - accuracy: 0.5570 - val_loss: 1.8041 - val_accuracy: 0.5000
Epoch 2/20
3/3 [==============================] - 12s 3s/step - loss: 179.7945 - accuracy: 0.6203 - val_loss: 0.7281 - val_accuracy: 0.5000
Epoch 3/20
3/3 [==============================] - 13s 3s/step - loss: 50.4149 - accuracy: 0.3924 - val_loss: 0.7047 - val_accuracy: 0.5000
Epoch 4/20
3/3 [==============================] - 12s 3s/step - loss: 19.2310 - accuracy: 0.5316 - val_loss: 0.6915 - val_accuracy: 0.5000
Epoch 5/20
3/3 [==============================] - 12s 3s/step - loss: 6.1660 - accuracy: 0.4430 - val_loss: 0.6929 - val_accuracy: 0.5000
Epoch 6/20
3/3 [==============================] - 13s 3s/step - loss: 0.6902 - accuracy: 0.5443 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 7/20
3/3 [==============================] - 24s 7s/step - loss: 0.6620 - accurac

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

# Define data paths and augmentations
train_data_dir = '/content/drive/My Drive/data/training'
validation_data_dir = '/content/drive/My Drive/data/validation'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Create and compile the model
def create_model(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.25),  # Add dropout regularization
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

model = create_model((224, 224, 3))

# Implement early stopping based on validation loss
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=20,  # Increase epochs for more training iterations
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=[early_stopping]
)

# Save the model
model.save('/content/drive/My Drive/model_optimized.h5')

# Evaluate the model
train_loss, train_accuracy = model.evaluate(train_generator)
print(f"Training Accuracy: {train_accuracy}")
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_accuracy}")


Found 79 images belonging to 2 classes.
Found 10 images belonging to 2 classes.
Epoch 1/20
3/3 [==============================] - 17s 4s/step - loss: 3.7319 - accuracy: 0.6203 - val_loss: 2.6365 - val_accuracy: 0.5000
Epoch 2/20
3/3 [==============================] - 13s 5s/step - loss: 1.5520 - accuracy: 0.3671 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 3/20
3/3 [==============================] - 13s 5s/step - loss: 0.6666 - accuracy: 0.6203 - val_loss: 0.7936 - val_accuracy: 0.5000
Epoch 4/20
3/3 [==============================] - 13s 3s/step - loss: 0.7054 - accuracy: 0.6329 - val_loss: 0.8065 - val_accuracy: 0.5000
Epoch 5/20
3/3 [==============================] - 6s 1s/step - loss: 0.6882 - accuracy: 0.6329
Training Accuracy: 0.6329113841056824
1/1 [==============================] - 1s 535ms/step - loss: 0.6933 - accuracy: 0.5000
Validation Accuracy: 0.5


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, concatenate

# Define a function to create the model with additional inputs for distance and angle
def create_model_with_features(input_shape, num_distances, num_angles):
    # Image input
    image_input = Input(shape=input_shape, name='image_input')
    x = Conv2D(32, (3, 3), activation='relu')(image_input)
    x = MaxPooling2D(2, 2)(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D(2, 2)(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D(2, 2)(x)
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)

    # Distance and angle inputs
    distance_input = Input(shape=(num_distances,), name='distance_input')
    angle_input = Input(shape=(num_angles,), name='angle_input')

    # Concatenate image features with distance and angle features
    combined = concatenate([x, distance_input, angle_input])

    # Dense layers for classification
    x = Dense(256, activation='relu')(combined)
    x = Dropout(0.5)(x)
    output = Dense(1, activation='sigmoid')(x)

    # Create model
    model = Model(inputs=[image_input, distance_input, angle_input], outputs=output)

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Example usage:
# Assuming you have extracted features (distances and angles) from your posture data
# and loaded your image data as X_images and Y_labels
X_images =
Y_labels =
X_distances =
X_angles =

# Define input shapes
image_input_shape = (224, 224, 3)  # Assuming image size is 224x224 and 3 channels (RGB)
num_distances = X_distances.shape[1]  # Number of distance features
num_angles = X_angles.shape[1]  # Number of angle features

# Create the model
model = create_model_with_features(image_input_shape, num_distances, num_angles)

# Train the model
history = model.fit(
    {'image_input': X_images, 'distance_input': X_distances, 'angle_input': X_angles},
    Y_labels,
    epochs=10,
    validation_split=0.2
)


In [ ]:
pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 16.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manyl

In [ ]:
import mediapipe as mp
import cv2
import numpy as np
import math

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

def calculate_features(image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)
    landmarks = results.pose_landmarks.landmark

    if not results.pose_landmarks:
        return None  # No keypoints detected

    # Define the required keypoints for feature calculation
    def get_xy(idx):
        return landmarks[idx].x, landmarks[idx].y

    l_shoulder = get_xy(mp_pose.PoseLandmark.LEFT_SHOULDER.value)
    r_shoulder = get_xy(mp_pose.PoseLandmark.RIGHT_SHOULDER.value)
    l_ear = get_xy(mp_pose.PoseLandmark.LEFT_EAR.value)
    l_hip = get_xy(mp_pose.PoseLandmark.LEFT_HIP.value)

    # Calculate distances and angles
    def find_distance(p1, p2):
        return math.sqrt((p2[0] - p1[0]) ** 2 + (p2[1] - p1[1]) ** 2)

    def find_angle(p1, p2):
        theta = math.acos((p2[1] - p1[1]) * (-p1[1]) / (math.sqrt((p2[0] - p1[0]) ** 2 + (p2[1] - p1[1]) ** 2) * p1[1]))
        degree = int(180 / math.pi) * theta
        return degree

    features = [
        find_distance(l_shoulder, r_shoulder),
        find_angle(l_shoulder, l_ear),
        find_angle(l_hip, l_shoulder)
    ]

    return features
def preprocess_dataset(image_paths):
    data = []
    labels = []  # Assume labels are given with paths

    for path, label in image_paths:
        image = cv2.imread(path)
        image = cv2.resize(image, (224, 224))
        features = calculate_features(image)
        if features:
            data.append(features)
            labels.append(label)

    return np.array(data), np.array(labels)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

def create_feature_model():
    model = Sequential([
        Dense(32, activation='relu', input_shape=(3,)),  # Input layer adjusted for 3 features
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = create_feature_model()
image_paths = [('/content/WIN_20240429_15_16_10_Pro.jpg', 0), ('/content/WIN_20240429_15_05_56_Pro.jpg', 1)]  # Example paths and labels
X, y = preprocess_dataset(image_paths)
model.fit(X, y, epochs=10, validation_split=0.2)



Epoch 1/10
1/1 [==============================] - 4s 4s/step - loss: 0.0916 - accuracy: 1.0000 - val_loss: 4.3875 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 87ms/step - loss: 0.0257 - accuracy: 1.0000 - val_loss: 4.6498 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 118ms/step - loss: 0.0314 - accuracy: 1.0000 - val_loss: 4.9315 - val_accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 0s 126ms/step - loss: 0.0325 - accuracy: 1.0000 - val_loss: 5.2103 - val_accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 0s 142ms/step - loss: 0.0365 - accuracy: 1.0000 - val_loss: 5.4876 - val_accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 0s 118ms/step - loss: 0.0376 - accuracy: 1.0000 - val_loss: 5.7706 - val_accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 0s 173ms/step - loss: 0.1067 - accuracy: 1.0000 - val_loss: 6.0403 - val_accuracy: 0.0000e

In [ ]:
validation_data = [('/content/WIN_20240429_15_16_07_Pro.jpg', 0), ('/content/WIN_20240429_15_05_56_Pro.jpg', 1)]
X_val, y_val = preprocess_dataset(validation_data)
accuracy = model.evaluate(X_val, y_val)
print("Validation Accuracy:", accuracy)
model.save('posture_model.h5')



1/1 [==============================] - 0s 26ms/step - loss: 3.1181 - accuracy: 0.5000
Validation Accuracy: [3.118060827255249, 0.5]


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
saved_model = tf.keras.models.load_model('posture_model.h5')
new_image_path = '/content/good_posture_032.jpg'
new_image = cv2.imread(new_image_path)
new_image_features = calculate_features(new_image)
if new_image_features:
    new_image_features = np.array([new_image_features])
    prediction = saved_model.predict(new_image_features)
    predicted_class = 'good' if prediction[0] > 0.5 else 'bad'
    print("Predicted Class:", predicted_class)
else:
    print("Could not detect keypoints in the image.")


1/1 [==============================] - 0s 64ms/step
Predicted Class: bad


TRYINGG

In [ ]:
import cv2
import math
import mediapipe as mp
import csv

mp_pose = mp.solutions.pose

def findDistance(x1, y1, x2, y2):
    dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return dist

def findAngle(x1, y1, x2, y2):
    theta = math.acos((y2 - y1) * (-y1) / (math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) * y1))
    degree = int(180 / math.pi) * theta
    return degree

good_frames = 0
bad_frames = 0

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

file_name = 'input.mp4'
cap = cv2.VideoCapture(file_name)

# Open CSV file for writing
with open('posture_data.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Left Shoulder X', 'Left Shoulder Y', 'Right Shoulder X', 'Right Shoulder Y',
                     'Left Ear X', 'Left Ear Y', 'Left Hip X', 'Left Hip Y', 'Neck Inclination', 'Torso Inclination', 'Label'])

    print('Processing..')
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Null.Frames")
            break

        h, w = image.shape[:2]
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        keypoints = pose.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        lm = keypoints.pose_landmarks
        lmPose = mp_pose.PoseLandmark

        if lm is not None:
            l_shldr_x = int(lm.landmark[lmPose.LEFT_SHOULDER].x * w)
            l_shldr_y = int(lm.landmark[lmPose.LEFT_SHOULDER].y * h)
            r_shldr_x = int(lm.landmark[lmPose.RIGHT_SHOULDER].x * w)
            r_shldr_y = int(lm.landmark[lmPose.RIGHT_SHOULDER].y * h)
            l_ear_x = int(lm.landmark[lmPose.LEFT_EAR].x * w)
            l_ear_y = int(lm.landmark[lmPose.LEFT_EAR].y * h)
            l_hip_x = int(lm.landmark[lmPose.LEFT_HIP].x * w)
            l_hip_y = int(lm.landmark[lmPose.LEFT_HIP].y * h)
            offset = findDistance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
            neck_inclination = findAngle(l_shldr_x, l_shldr_y, l_ear_x, l_ear_y)
            torso_inclination = findAngle(l_hip_x, l_hip_y, l_shldr_x, l_shldr_y)

            # Determine label based on posture
            label = 'good' if offset < 100 and neck_inclination < 40 and torso_inclination < 10 else 'bad'

            # Write posture data to CSV file
            writer.writerow([l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y,
                             l_ear_x, l_ear_y, l_hip_x, l_hip_y,
                             neck_inclination, torso_inclination, label])

print('Finished.')
cap.release()


Processing..
Finished.


In [ ]:

folder_path = '/content/drive/MyDrive/data'  # Adjust the path to where your images are stored within Google Drive
csv_file_path = '/content/drive/My Drive/posture_data1.csv'  # Adjust the path where you want to save the CSV


In [ ]:
import os
import cv2
import math
import mediapipe as mp
import csv

mp_pose = mp.solutions.pose

def findDistance(x1, y1, x2, y2):
    dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return dist

def findAngle(x1, y1, x2, y2):
    if (x2 - x1) == 0 or y1 == 0:  # Avoid division by zero
        return 0
    theta = math.acos((y2 - y1) * (-y1) / (math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) * y1))
    degree = int(180 / math.pi) * theta
    return degree

def process_image(image_path, writer, pose):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to read image from {image_path}")
        return

    h, w = image.shape[:2]
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    keypoints = pose.process(image_rgb)
    lm = keypoints.pose_landmarks

    if lm is not None:
        l_shldr_x = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
        l_shldr_y = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
        r_shldr_x = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
        r_shldr_y = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
        l_ear_x = lm.landmark[mp_pose.PoseLandmark.LEFT_EAR].x * w
        l_ear_y = lm.landmark[mp_pose.PoseLandmark.LEFT_EAR].y * h
        l_hip_x = lm.landmark[mp_pose.PoseLandmark.LEFT_HIP].x * w
        l_hip_y = lm.landmark[mp_pose.PoseLandmark.LEFT_HIP].y * h
        offset = findDistance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        neck_inclination = findAngle(l_shldr_x, l_shldr_y, l_ear_x, l_ear_y)
        torso_inclination = findAngle(l_hip_x, l_hip_y, l_shldr_x, l_shldr_y)

        # Determine label based on posture
        label = 'good' if offset < 100 and neck_inclination < 40 and torso_inclination < 10 else 'bad'

        # Write posture data to CSV file
        writer.writerow([l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y,
                         l_ear_x, l_ear_y, l_hip_x, l_hip_y,
                         neck_inclination, torso_inclination, label])
    else:
        print(f"No landmarks detected in image {image_path}")

def process_images_in_folder(folder_path, csv_file_path):
    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Left Shoulder X', 'Left Shoulder Y', 'Right Shoulder X', 'Right Shoulder Y',
                         'Left Ear X', 'Left Ear Y', 'Left Hip X', 'Left Hip Y', 'Neck Inclination',
                         'Torso Inclination', 'Label'])

        pose = mp_pose.Pose()
        for filename in os.listdir(folder_path):
            if filename.endswith(".jpg") or filename.endswith(".png"):
                image_path = os.path.join(folder_path, filename)
                print("Processing:", image_path)  # Add this line for debugging
                process_image(image_path, writer, pose)
            else:
                print("Skipping file:", filename)  # Add this line for debugging
        pose.close()

if __name__ == "__main__":
    folder_path = '/content/drive/MyDrive/data'  # Adjust the path to where your images are stored within Google Drive
    csv_file_path = '/content/posture_data3.csv'
    process_images_in_folder(folder_path, csv_file_path)
    print('Finished processing images and saving data to CSV.')


Skipping file: validation
Skipping file: training
Finished processing images and saving data to CSV.


In [ ]:
import os
import cv2
import math
import mediapipe as mp
import csv

mp_pose = mp.solutions.pose

def findDistance(x1, y1, x2, y2):
    dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return dist

def findAngle(x1, y1, x2, y2):
    if (x2 - x1) == 0 or y1 == 0:
        return 0
    theta = math.acos((y2 - y1) * (-y1) / (math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) * y1))
    degree = int(180 / math.pi) * theta
    return degree

def process_image(image_path, writer, pose):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to read image from {image_path}")
        return

    h, w = image.shape[:2]
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    keypoints = pose.process(image_rgb)
    lm = keypoints.pose_landmarks

    if lm is not None:
        l_shldr_x = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
        l_shldr_y = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
        r_shldr_x = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
        r_shldr_y = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
        l_ear_x = lm.landmark[mp_pose.PoseLandmark.LEFT_EAR].x * w
        l_ear_y = lm.landmark[mp_pose.PoseLandmark.LEFT_EAR].y * h
        l_hip_x = lm.landmark[mp_pose.PoseLandmark.LEFT_HIP].x * w
        l_hip_y = lm.landmark[mp_pose.PoseLandmark.LEFT_HIP].y * h
        offset = findDistance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        neck_inclination = findAngle(l_shldr_x, l_shldr_y, l_ear_x, l_ear_y)
        torso_inclination = findAngle(l_hip_x, l_hip_y, l_shldr_x, l_shldr_y)

        # Determine label based on posture
        label = 'good' if offset < 100 and neck_inclination < 40 and torso_inclination < 10 else 'bad'

        # Write posture data to CSV file
        writer.writerow([l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y,
                         l_ear_x, l_ear_y, l_hip_x, l_hip_y,
                         neck_inclination, torso_inclination, label])
    else:
        print(f"No landmarks detected in image {image_path}")

def process_images_recursively(folder_path, writer, pose):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.lower().endswith(".jpg") or filename.lower().endswith(".png"):
                image_path = os.path.join(root, filename)
                print("Processing:", image_path)
                process_image(image_path, writer, pose)

def main():
    folder_path = '/content/drive/MyDrive/data'
    csv_file_path = '/content/posture_dataA.csv'

    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Left Shoulder X', 'Left Shoulder Y','Right Shoulder X' ,'Right Shoulder Y','','',''])

        pose = mp_pose.Pose()
        process_images_recursively(folder_path, writer, pose)
        pose.close()

    print('Finished processing images and saving data to CSV.')

if __name__ == "__main__":
    main()


Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_003.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_002.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_005.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_001.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_004.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_007.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_008.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_006.jpg
Processing: /content/drive/MyDrive/data/validation/bad/bad_posture_val_001.jpg
Processing: /content/drive/MyDrive/data/validation/bad/bad_posture_val_002.jpg
Processing: /content/drive/MyDrive/data/validation/bad/bad_posture_val_003.jpg
Processing: /content/drive/MyDrive/data/validation/bad/bad_posture_val_004.jpg
Processing: /content/drive/MyDrive/d

In [ ]:
import os
import cv2
import math
import mediapipe as mp
import csv

mp_pose = mp.solutions.pose

def findDistance(x1, y1, x2, y2):
    dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return dist

def findAngle(x1, y1, x2, y2):
    if (x2 - x1) == 0 or y1 == 0:
        return 0
    theta = math.acos((y2 - y1) * (-y1) / (math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) * y1))
    degree = int(180 / math.pi) * theta
    return degree

def process_image(image_path, writer, pose):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to read image from {image_path}")
        return

    h, w = image.shape[:2]
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    keypoints = pose.process(image_rgb)
    lm = keypoints.pose_landmarks

    if lm is not None:
        nose_x = lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
        nose_y = lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
        nose_z = lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

        l_shldr_x = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
        l_shldr_y = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
        l_shldr_z = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
        r_shldr_x = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
        r_shldr_y = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
        r_shldr_z=lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
        #offset = findDistance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        #nose_angle = findAngle(l_shldr_x, l_shldr_y, nose_x, nose_y)
        #torso_inclination = findAngle(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)

        label = 'good'
        # Write posture data to CSV file
        writer.writerow([nose_x, nose_y, nose_z, l_shldr_x,l_shldr_y,l_shldr_z,r_shldr_x,r_shldr_y,r_shldr_z,label])
    else:
        print(f"No landmarks detected in image {image_path}")

def process_images_recursively(folder_path, writer, pose):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.lower().endswith(".jpg") or filename.lower().endswith(".png"):
                image_path = os.path.join(root, filename)
                print("Processing:", image_path)
                process_image(image_path, writer, pose)

def main():
    folder_path = '/content/drive/MyDrive/data/training/good'
    csv_file_path = '/content/posture_dataB.csv'

    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Nose X', 'Nose Y', 'Nose Z','Left Shoulder X', 'Left Shoulder Y','Left Shoulder Z','Right Shoulder X' ,'Right Shoulder Y', 'Right Shoulder Z','Label'])

        pose = mp_pose.Pose()
        process_images_recursively(folder_path, writer, pose)
        pose.close()

    print('Finished processing images and saving data to CSV.')

if __name__ == "__main__":
    main()


Processing: /content/drive/MyDrive/data/training/good/good_posture_026.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_024.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_029.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_027.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_028.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_023 (1).jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_022.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_025.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_021.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_018.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_019.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_020.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_017.jpg
Processing: /content/

In [ ]:
import os
import cv2
import math
import mediapipe as mp
import csv

mp_pose = mp.solutions.pose

def findDistance(x1, y1, x2, y2):
    dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return dist

def findAngle(x1, y1, x2, y2):
    if (x2 - x1) == 0 or y1 == 0:
        return 0
    theta = math.acos((y2 - y1) * (-y1) / (math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) * y1))
    degree = int(180 / math.pi) * theta
    return degree

def process_image(image_path, writer, pose):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to read image from {image_path}")
        return

    h, w = image.shape[:2]
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    keypoints = pose.process(image_rgb)
    lm = keypoints.pose_landmarks

    if lm is not None:
        nose_x = lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
        nose_y = lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
        nose_z = lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

        l_shldr_x = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
        l_shldr_y = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
        l_shldr_z = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
        r_shldr_x = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
        r_shldr_y = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
        r_shldr_z=lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
        #offset = findDistance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        #nose_angle = findAngle(l_shldr_x, l_shldr_y, nose_x, nose_y)
        #torso_inclination = findAngle(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)

        label = 'bad'
        # Write posture data to CSV file
        writer.writerow([nose_x, nose_y, nose_z, l_shldr_x,l_shldr_y,l_shldr_z,r_shldr_x,r_shldr_y,r_shldr_z,label])
    else:
        print(f"No landmarks detected in image {image_path}")

def process_images_recursively(folder_path, writer, pose):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.lower().endswith(".jpg") or filename.lower().endswith(".png"):
                image_path = os.path.join(root, filename)
                print("Processing:", image_path)
                process_image(image_path, writer, pose)

def main():
    folder_path = '/content/drive/MyDrive/data/training/bad'
    csv_file_path = '/content/badtraining.csv'

    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Nose X', 'Nose Y', 'Nose Z','Left Shoulder X', 'Left Shoulder Y','Left Shoulder Z','Right Shoulder X' ,'Right Shoulder Y', 'Right Shoulder Z','Label'])

        pose = mp_pose.Pose()
        process_images_recursively(folder_path, writer, pose)
        pose.close()

    print('Finished processing images and saving data to CSV.')

if __name__ == "__main__":
    main()


Processing: /content/drive/MyDrive/data/training/bad/bad_posture_002.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_046.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_003.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_013.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_031.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_026.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_020.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_041.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_019.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_036.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_005.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_014.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_008.jpg
Processing: /content/drive/MyDrive/data/training/ba

In [ ]:
import os
import cv2
import math
import mediapipe as mp
import csv

mp_pose = mp.solutions.pose

def findDistance(x1, y1, x2, y2):
    dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return dist

def findAngle(x1, y1, x2, y2):
    if (x2 - x1) == 0 or y1 == 0:
        return 0
    theta = math.acos((y2 - y1) * (-y1) / (math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) * y1))
    degree = int(180 / math.pi) * theta
    return degree

def process_image(image_path, writer, pose):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to read image from {image_path}")
        return

    h, w = image.shape[:2]
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    keypoints = pose.process(image_rgb)
    lm = keypoints.pose_landmarks

    if lm is not None:
        nose_x = lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
        nose_y = lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
        nose_z = lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

        l_shldr_x = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
        l_shldr_y = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
        l_shldr_z = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
        r_shldr_x = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
        r_shldr_y = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
        r_shldr_z=lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
        #offset = findDistance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        #nose_angle = findAngle(l_shldr_x, l_shldr_y, nose_x, nose_y)
        #torso_inclination = findAngle(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)

        label = 'bad'
        # Write posture data to CSV file
        writer.writerow([nose_x, nose_y, nose_z, l_shldr_x,l_shldr_y,l_shldr_z,r_shldr_x,r_shldr_y,r_shldr_z,label])
    else:
        print(f"No landmarks detected in image {image_path}")

def process_images_recursively(folder_path, writer, pose):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.lower().endswith(".jpg") or filename.lower().endswith(".png"):
                image_path = os.path.join(root, filename)
                print("Processing:", image_path)
                process_image(image_path, writer, pose)

def main():
    folder_path = '/content/drive/MyDrive/data/validation/bad'
    csv_file_path = '/content/badvalidation.csv'

    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Nose X', 'Nose Y', 'Nose Z','Left Shoulder X', 'Left Shoulder Y','Left Shoulder Z','Right Shoulder X' ,'Right Shoulder Y', 'Right Shoulder Z','Label'])

        pose = mp_pose.Pose()
        process_images_recursively(folder_path, writer, pose)
        pose.close()

    print('Finished processing images and saving data to CSV.')

if __name__ == "__main__":
    main()


Processing: /content/drive/MyDrive/data/validation/bad/bad_posture_val_001.jpg
Processing: /content/drive/MyDrive/data/validation/bad/bad_posture_val_002.jpg
Processing: /content/drive/MyDrive/data/validation/bad/bad_posture_val_003.jpg
Processing: /content/drive/MyDrive/data/validation/bad/bad_posture_val_004.jpg
Processing: /content/drive/MyDrive/data/validation/bad/bad_posture_val_005.jpg
Processing: /content/drive/MyDrive/data/validation/bad/bad_posture_val_006.jpg
Processing: /content/drive/MyDrive/data/validation/bad/bad_posture_val_007.jpg
Processing: /content/drive/MyDrive/data/validation/bad/bad_posture_val_008.jpg
Finished processing images and saving data to CSV.


In [ ]:
import os
import cv2
import math
import mediapipe as mp
import csv

mp_pose = mp.solutions.pose

def findDistance(x1, y1, x2, y2):
    dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return dist

def findAngle(x1, y1, x2, y2):
    if (x2 - x1) == 0 or y1 == 0:
        return 0
    theta = math.acos((y2 - y1) * (-y1) / (math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) * y1))
    degree = int(180 / math.pi) * theta
    return degree

def process_image(image_path, writer, pose):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to read image from {image_path}")
        return

    h, w = image.shape[:2]
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    keypoints = pose.process(image_rgb)
    lm = keypoints.pose_landmarks

    if lm is not None:
        nose_x = lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
        nose_y = lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
        nose_z = lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

        l_shldr_x = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
        l_shldr_y = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
        l_shldr_z = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
        r_shldr_x = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
        r_shldr_y = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
        r_shldr_z=lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
        #offset = findDistance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        #nose_angle = findAngle(l_shldr_x, l_shldr_y, nose_x, nose_y)
        #torso_inclination = findAngle(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)

        label = 'good'
        # Write posture data to CSV file
        writer.writerow([nose_x, nose_y, nose_z, l_shldr_x,l_shldr_y,l_shldr_z,r_shldr_x,r_shldr_y,r_shldr_z,label])
    else:
        print(f"No landmarks detected in image {image_path}")

def process_images_recursively(folder_path, writer, pose):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.lower().endswith(".jpg") or filename.lower().endswith(".png"):
                image_path = os.path.join(root, filename)
                print("Processing:", image_path)
                process_image(image_path, writer, pose)

def main():
    folder_path = '/content/drive/MyDrive/data/validation/good'
    csv_file_path = '/content/goodvalidation.csv'

    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Nose X', 'Nose Y', 'Nose Z','Left Shoulder X', 'Left Shoulder Y','Left Shoulder Z','Right Shoulder X' ,'Right Shoulder Y', 'Right Shoulder Z','Label'])

        pose = mp_pose.Pose()
        process_images_recursively(folder_path, writer, pose)
        pose.close()

    print('Finished processing images and saving data to CSV.')

if __name__ == "__main__":
    main()


Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_003.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_002.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_005.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_001.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_004.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_007.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_008.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_006.jpg
Finished processing images and saving data to CSV.


In [ ]:
import os
import cv2
import math
import mediapipe as mp
import csv

mp_pose = mp.solutions.pose

def findDistance(x1, y1, x2, y2):
    dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return dist

def findAngle(x1, y1, x2, y2):
    if (x2 - x1) == 0 or y1 == 0:
        return 0
    theta = math.acos((y2 - y1) * (-y1) / (math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) * y1))
    degree = int(180 / math.pi) * theta
    return degree

def process_image(image_path, writer, pose):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to read image from {image_path}")
        return

    h, w = image.shape[:2]
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    keypoints = pose.process(image_rgb)
    lm = keypoints.pose_landmarks

    if lm is not None:
        nose_x = lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
        nose_y = lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
        nose_z = lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

        l_shldr_x = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
        l_shldr_y = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
        l_shldr_z = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
        r_shldr_x = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
        r_shldr_y = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
        r_shldr_z=lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
        #offset = findDistance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        #nose_angle = findAngle(l_shldr_x, l_shldr_y, nose_x, nose_y)
        #torso_inclination = findAngle(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)

        label = 'good'
        # Write posture data to CSV file
        writer.writerow([nose_x, nose_y, nose_z, l_shldr_x,l_shldr_y,l_shldr_z,r_shldr_x,r_shldr_y,r_shldr_z,label])
    else:
        print(f"No landmarks detected in image {image_path}")

def process_images_recursively(folder_path, writer, pose):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.lower().endswith(".jpg") or filename.lower().endswith(".png"):
                image_path = os.path.join(root, filename)
                print("Processing:", image_path)
                process_image(image_path, writer, pose)

def main():
    folder_path = '/content/drive/MyDrive/data/training/good'
    csv_file_path = '/content/goodtr.csv'

    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Nose X', 'Nose Y', 'Nose Z','Left Shoulder X', 'Left Shoulder Y','Left Shoulder Z','Right Shoulder X' ,'Right Shoulder Y', 'Right Shoulder Z','Label'])

        pose = mp_pose.Pose()
        process_images_recursively(folder_path, writer, pose)
        pose.close()

    print('Finished processing images and saving data to CSV.')

if __name__ == "__main__":
    main()


Processing: /content/drive/MyDrive/data/training/good/good_posture_026.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_024.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_029.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_027.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_028.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_023 (1).jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_022.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_025.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_021.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_018.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_019.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_020.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_017.jpg
Processing: /content/

In [ ]:
import os
import cv2
import math
import mediapipe as mp
import csv

mp_pose = mp.solutions.pose

def findDistance(x1, y1, x2, y2):
    dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return dist

def findAngle(x1, y1, x2, y2):
    if (x2 - x1) == 0 or y1 == 0:
        return 0
    theta = math.acos((y2 - y1) * (-y1) / (math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) * y1))
    degree = int(180 / math.pi) * theta
    return degree

def process_image(image_path, writer, pose):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to read image from {image_path}")
        return

    h, w = image.shape[:2]
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    keypoints = pose.process(image_rgb)
    lm = keypoints.pose_landmarks

    if lm is not None:
        nose_x = lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
        nose_y = lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
        nose_z = lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

        l_shldr_x = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
        l_shldr_y = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
        l_shldr_z = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
        r_shldr_x = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
        r_shldr_y = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
        r_shldr_z=lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
        #offset = findDistance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        #nose_angle = findAngle(l_shldr_x, l_shldr_y, nose_x, nose_y)
        #torso_inclination = findAngle(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)

        label = 'bad'
        # Write posture data to CSV file
        writer.writerow([nose_x, nose_y, nose_z, l_shldr_x,l_shldr_y,l_shldr_z,r_shldr_x,r_shldr_y,r_shldr_z,label])
    else:
        print(f"No landmarks detected in image {image_path}")

def process_images_recursively(folder_path, writer, pose):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.lower().endswith(".jpg") or filename.lower().endswith(".png"):
                image_path = os.path.join(root, filename)
                print("Processing:", image_path)
                process_image(image_path, writer, pose)

def main():
    folder_path = '/content/drive/MyDrive/data/training/bad'
    csv_file_path = '/content/badtr.csv'

    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Nose X', 'Nose Y', 'Nose Z','Left Shoulder X', 'Left Shoulder Y','Left Shoulder Z','Right Shoulder X' ,'Right Shoulder Y', 'Right Shoulder Z','Label'])

        pose = mp_pose.Pose()
        process_images_recursively(folder_path, writer, pose)
        pose.close()

    print('Finished processing images and saving data to CSV.')

if __name__ == "__main__":
    main()


Processing: /content/drive/MyDrive/data/training/bad/bad_posture_002.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_046.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_003.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_013.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_031.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_026.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_020.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_041.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_019.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_036.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_005.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_014.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_008.jpg
Processing: /content/drive/MyDrive/data/training/ba

In [ ]:
import os
import cv2
import math
import mediapipe as mp
import csv

mp_pose = mp.solutions.pose

def findDistance(x1, y1, x2, y2):
    dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return dist

def findAngle(x1, y1, x2, y2):
    if (x2 - x1) == 0 or y1 == 0:
        return 0
    theta = math.acos((y2 - y1) * (-y1) / (math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) * y1))
    degree = int(180 / math.pi) * theta
    return degree

def process_image(image_path, writer, pose):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to read image from {image_path}")
        return

    h, w = image.shape[:2]
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    keypoints = pose.process(image_rgb)
    lm = keypoints.pose_landmarks

    if lm is not None:
        nose_x = lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
        nose_y = lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
        nose_z = lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

        l_shldr_x = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
        l_shldr_y = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
        l_shldr_z = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
        r_shldr_x = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
        r_shldr_y = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
        r_shldr_z=lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
        #offset = findDistance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        #nose_angle = findAngle(l_shldr_x, l_shldr_y, nose_x, nose_y)
        #torso_inclination = findAngle(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)

        label = 'bad'
        # Write posture data to CSV file
        writer.writerow([nose_x, nose_y, nose_z, l_shldr_x,l_shldr_y,l_shldr_z,r_shldr_x,r_shldr_y,r_shldr_z,label])
    else:
        print(f"No landmarks detected in image {image_path}")

def process_images_recursively(folder_path, writer, pose):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.lower().endswith(".jpg") or filename.lower().endswith(".png"):
                image_path = os.path.join(root, filename)
                print("Processing:", image_path)
                process_image(image_path, writer, pose)

def main():
    folder_path = '/content/drive/MyDrive/data/validation/bad'
    csv_file_path = '/content/badval.csv'

    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Nose X', 'Nose Y', 'Nose Z','Left Shoulder X', 'Left Shoulder Y','Left Shoulder Z','Right Shoulder X' ,'Right Shoulder Y', 'Right Shoulder Z','Label'])

        pose = mp_pose.Pose()
        process_images_recursively(folder_path, writer, pose)
        pose.close()

    print('Finished processing images and saving data to CSV.')

if __name__ == "__main__":
    main()


Processing: /content/drive/MyDrive/data/validation/bad/bad_posture_val_001.jpg
Processing: /content/drive/MyDrive/data/validation/bad/bad_posture_val_002.jpg
Processing: /content/drive/MyDrive/data/validation/bad/bad_posture_val_003.jpg
Processing: /content/drive/MyDrive/data/validation/bad/bad_posture_val_004.jpg
Processing: /content/drive/MyDrive/data/validation/bad/bad_posture_val_005.jpg
Processing: /content/drive/MyDrive/data/validation/bad/bad_posture_val_006.jpg
Processing: /content/drive/MyDrive/data/validation/bad/bad_posture_val_007.jpg
Processing: /content/drive/MyDrive/data/validation/bad/bad_posture_val_008.jpg
Processing: /content/drive/MyDrive/data/validation/bad/WIN_20240429_19_26_47_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/bad/WIN_20240429_19_24_52_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/bad/WIN_20240429_19_24_53_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/bad/WIN_20240429_19_24_54_Pro.jpg
Processing: /content/drive/M

In [ ]:
import os
import cv2
import math
import mediapipe as mp
import csv

mp_pose = mp.solutions.pose

def findDistance(x1, y1, x2, y2):
    dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return dist

def findAngle(x1, y1, x2, y2):
    if (x2 - x1) == 0 or y1 == 0:
        return 0
    theta = math.acos((y2 - y1) * (-y1) / (math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) * y1))
    degree = int(180 / math.pi) * theta
    return degree

def process_image(image_path, writer, pose):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to read image from {image_path}")
        return

    h, w = image.shape[:2]
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    keypoints = pose.process(image_rgb)
    lm = keypoints.pose_landmarks

    if lm is not None:
        nose_x = lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
        nose_y = lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
        nose_z = lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

        l_shldr_x = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
        l_shldr_y = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
        l_shldr_z = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
        r_shldr_x = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
        r_shldr_y = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
        r_shldr_z=lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
        #offset = findDistance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        #nose_angle = findAngle(l_shldr_x, l_shldr_y, nose_x, nose_y)
        #torso_inclination = findAngle(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)

        label = 'good'
        # Write posture data to CSV file
        writer.writerow([nose_x, nose_y, nose_z, l_shldr_x,l_shldr_y,l_shldr_z,r_shldr_x,r_shldr_y,r_shldr_z,label])
    else:
        print(f"No landmarks detected in image {image_path}")

def process_images_recursively(folder_path, writer, pose):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.lower().endswith(".jpg") or filename.lower().endswith(".png"):
                image_path = os.path.join(root, filename)
                print("Processing:", image_path)
                process_image(image_path, writer, pose)

def main():
    folder_path = '/content/drive/MyDrive/data/validation/good'
    csv_file_path = '/content/goodval.csv'

    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Nose X', 'Nose Y', 'Nose Z','Left Shoulder X', 'Left Shoulder Y','Left Shoulder Z','Right Shoulder X' ,'Right Shoulder Y', 'Right Shoulder Z','Label'])

        pose = mp_pose.Pose()
        process_images_recursively(folder_path, writer, pose)
        pose.close()

    print('Finished processing images and saving data to CSV.')

if __name__ == "__main__":
    main()


Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_003.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_002.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_005.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_001.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_004.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_007.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_006.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_19_25_32_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_19_25_33_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_19_24_44_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_19_24_46_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_19_24_47_Pro.jpg
Pro

In [ ]:
pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 23.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manyl

In [ ]:
import os
import cv2
import math
import mediapipe as mp
import csv

mp_pose = mp.solutions.pose

def findDistance(x1, y1, x2, y2):
    dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return dist

def findAngle(x1, y1, x2, y2):
    if (x2 - x1) == 0 or y1 == 0:
        return 0
    theta = math.acos((y2 - y1) * (-y1) / (math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) * y1))
    degree = int(180 / math.pi) * theta
    return degree

def process_image(image_path, writer, pose):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to read image from {image_path}")
        return

    h, w = image.shape[:2]
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    keypoints = pose.process(image_rgb)
    lm = keypoints.pose_landmarks

    if lm is not None:
        nose_x = lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
        nose_y = lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
        nose_z = lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

        l_shldr_x = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
        l_shldr_y = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
        l_shldr_z = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
        r_shldr_x = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
        r_shldr_y = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
        r_shldr_z=lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
        #offset = findDistance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        #nose_angle = findAngle(l_shldr_x, l_shldr_y, nose_x, nose_y)
        #torso_inclination = findAngle(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)

        label = 'good'
        # Write posture data to CSV file
        writer.writerow([nose_x, nose_y, nose_z, l_shldr_x,l_shldr_y,l_shldr_z,r_shldr_x,r_shldr_y,r_shldr_z,label])
    else:
        print(f"No landmarks detected in image {image_path}")

def process_images_recursively(folder_path, writer, pose):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.lower().endswith(".jpg") or filename.lower().endswith(".png"):
                image_path = os.path.join(root, filename)
                print("Processing:", image_path)
                process_image(image_path, writer, pose)

def main():
    folder_path = '/content/drive/MyDrive/data/training/good'
    csv_file_path = '/content/gootr.csv'

    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Nose X', 'Nose Y', 'Nose Z','Left Shoulder X', 'Left Shoulder Y','Left Shoulder Z','Right Shoulder X' ,'Right Shoulder Y', 'Right Shoulder Z','Label'])

        pose = mp_pose.Pose()
        process_images_recursively(folder_path, writer, pose)
        pose.close()

    print('Finished processing images and saving data to CSV.')

if __name__ == "__main__":
    main()


Processing: /content/drive/MyDrive/data/training/good/WIN_20240429_22_28_19_Pro.jpg
Processing: /content/drive/MyDrive/data/training/good/WIN_20240429_22_27_54_Pro.jpg
Processing: /content/drive/MyDrive/data/training/good/WIN_20240429_22_27_53_Pro.jpg
Processing: /content/drive/MyDrive/data/training/good/WIN_20240429_22_28_18_Pro.jpg
Processing: /content/drive/MyDrive/data/training/good/WIN_20240429_22_29_17_Pro.jpg
Processing: /content/drive/MyDrive/data/training/good/WIN_20240429_22_29_05_Pro.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_001.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_002.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_003.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_004.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_005.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_006.jpg
Processing: /content/drive/MyDrive/data/traini

In [ ]:
import os
import cv2
import math
import mediapipe as mp
import csv

mp_pose = mp.solutions.pose

def findDistance(x1, y1, x2, y2):
    dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return dist

def findAngle(x1, y1, x2, y2):
    if (x2 - x1) == 0 or y1 == 0:
        return 0
    theta = math.acos((y2 - y1) * (-y1) / (math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) * y1))
    degree = int(180 / math.pi) * theta
    return degree

def process_image(image_path, writer, pose):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to read image from {image_path}")
        return

    h, w = image.shape[:2]
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    keypoints = pose.process(image_rgb)
    lm = keypoints.pose_landmarks

    if lm is not None:
        nose_x = lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
        nose_y = lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
        nose_z = lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

        l_shldr_x = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
        l_shldr_y = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
        l_shldr_z = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
        r_shldr_x = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
        r_shldr_y = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
        r_shldr_z=lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
        #offset = findDistance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        #nose_angle = findAngle(l_shldr_x, l_shldr_y, nose_x, nose_y)
        #torso_inclination = findAngle(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)

        label = 'bad'
        # Write posture data to CSV file
        writer.writerow([nose_x, nose_y, nose_z, l_shldr_x,l_shldr_y,l_shldr_z,r_shldr_x,r_shldr_y,r_shldr_z,label])
    else:
        print(f"No landmarks detected in image {image_path}")

def process_images_recursively(folder_path, writer, pose):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.lower().endswith(".jpg") or filename.lower().endswith(".png"):
                image_path = os.path.join(root, filename)
                print("Processing:", image_path)
                process_image(image_path, writer, pose)

def main():
    folder_path = '/content/drive/MyDrive/data/training/bad'
    csv_file_path = '/content/batr.csv'

    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Nose X', 'Nose Y', 'Nose Z','Left Shoulder X', 'Left Shoulder Y','Left Shoulder Z','Right Shoulder X' ,'Right Shoulder Y', 'Right Shoulder Z','Label'])

        pose = mp_pose.Pose()
        process_images_recursively(folder_path, writer, pose)
        pose.close()

    print('Finished processing images and saving data to CSV.')

if __name__ == "__main__":
    main()


Processing: /content/drive/MyDrive/data/training/bad/bad_posture_001.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_002.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_003.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_004.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_005.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_006.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_007.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_008.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_009.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_010.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_011.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_012.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_013.jpg
Processing: /content/drive/MyDrive/data/training/ba

In [ ]:
import os
import cv2
import math
import mediapipe as mp
import csv

mp_pose = mp.solutions.pose

def findDistance(x1, y1, x2, y2):
    dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return dist

def findAngle(x1, y1, x2, y2):
    if (x2 - x1) == 0 or y1 == 0:
        return 0
    theta = math.acos((y2 - y1) * (-y1) / (math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) * y1))
    degree = int(180 / math.pi) * theta
    return degree

def process_image(image_path, writer, pose):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to read image from {image_path}")
        return

    h, w = image.shape[:2]
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    keypoints = pose.process(image_rgb)
    lm = keypoints.pose_landmarks

    if lm is not None:
        nose_x = lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
        nose_y = lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
        nose_z = lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

        l_shldr_x = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
        l_shldr_y = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
        l_shldr_z = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
        r_shldr_x = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
        r_shldr_y = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
        r_shldr_z=lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
        #offset = findDistance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        #nose_angle = findAngle(l_shldr_x, l_shldr_y, nose_x, nose_y)
        #torso_inclination = findAngle(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)

        label = 'bad'
        # Write posture data to CSV file
        writer.writerow([nose_x, nose_y, nose_z, l_shldr_x,l_shldr_y,l_shldr_z,r_shldr_x,r_shldr_y,r_shldr_z,label])
    else:
        print(f"No landmarks detected in image {image_path}")

def process_images_recursively(folder_path, writer, pose):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.lower().endswith(".jpg") or filename.lower().endswith(".png"):
                image_path = os.path.join(root, filename)
                print("Processing:", image_path)
                process_image(image_path, writer, pose)

def main():
    folder_path = '/content/drive/MyDrive/data/validation/bad'
    csv_file_path = '/content/baval.csv'

    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Nose X', 'Nose Y', 'Nose Z','Left Shoulder X', 'Left Shoulder Y','Left Shoulder Z','Right Shoulder X' ,'Right Shoulder Y', 'Right Shoulder Z','Label'])

        pose = mp_pose.Pose()
        process_images_recursively(folder_path, writer, pose)
        pose.close()

    print('Finished processing images and saving data to CSV.')

if __name__ == "__main__":
    main()


Processing: /content/drive/MyDrive/data/validation/bad/WIN_20240429_22_28_52_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/bad/WIN_20240429_22_27_56_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/bad/WIN_20240429_22_27_49_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/bad/WIN_20240429_22_28_21_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/bad/WIN_20240429_22_27_41_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/bad/WIN_20240429_22_29_13_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/bad/bad_posture_val_001.jpg
Processing: /content/drive/MyDrive/data/validation/bad/bad_posture_val_002.jpg
Processing: /content/drive/MyDrive/data/validation/bad/bad_posture_val_003.jpg
Processing: /content/drive/MyDrive/data/validation/bad/bad_posture_val_004.jpg
Processing: /content/drive/MyDrive/data/validation/bad/bad_posture_val_005.jpg
Processing: /content/drive/MyDrive/data/validation/bad/bad_posture_val_006.jpg
Processing: /con

In [ ]:
import os
import cv2
import math
import mediapipe as mp
import csv

mp_pose = mp.solutions.pose

def findDistance(x1, y1, x2, y2):
    dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return dist

def findAngle(x1, y1, x2, y2):
    if (x2 - x1) == 0 or y1 == 0:
        return 0
    theta = math.acos((y2 - y1) * (-y1) / (math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) * y1))
    degree = int(180 / math.pi) * theta
    return degree

def process_image(image_path, writer, pose):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to read image from {image_path}")
        return

    h, w = image.shape[:2]
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    keypoints = pose.process(image_rgb)
    lm = keypoints.pose_landmarks

    if lm is not None:
        nose_x = lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
        nose_y = lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
        nose_z = lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

        l_shldr_x = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
        l_shldr_y = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
        l_shldr_z = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
        r_shldr_x = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
        r_shldr_y = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
        r_shldr_z=lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
        #offset = findDistance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        #nose_angle = findAngle(l_shldr_x, l_shldr_y, nose_x, nose_y)
        #torso_inclination = findAngle(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        l_shldr_x -= r_shldr_x
        l_shldr_y -= r_shldr_y
        nose_x -= r_shldr_x
        nose_y -= r_shldr_y
        r_shldr_x = 0
        r_shldr_y = 0

        label = 'good'
        # Write posture data to CSV file
        writer.writerow([nose_x, nose_y, nose_z, l_shldr_x,l_shldr_y,l_shldr_z,r_shldr_x,r_shldr_y,r_shldr_z,label])
    else:
        print(f"No landmarks detected in image {image_path}")

def process_images_recursively(folder_path, writer, pose):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.lower().endswith(".jpg") or filename.lower().endswith(".png"):
                image_path = os.path.join(root, filename)
                print("Processing:", image_path)
                process_image(image_path, writer, pose)

def main():
    folder_path = '/content/drive/MyDrive/data/validation/good'
    csv_file_path = '/content/gooval.csv'

    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Nose X', 'Nose Y', 'Nose Z','Left Shoulder X', 'Left Shoulder Y','Left Shoulder Z','Right Shoulder X' ,'Right Shoulder Y', 'Right Shoulder Z','Label'])

        pose = mp_pose.Pose()
        process_images_recursively(folder_path, writer, pose)
        pose.close()

    print('Finished processing images and saving data to CSV.')

if __name__ == "__main__":
    main()


Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_22_28_19_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_22_28_18_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_22_29_17_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_22_27_53_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_22_27_54_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_22_29_05_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_001.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_002.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_003.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_004.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_005.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_006.jp

In [ ]:
pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 31.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manyl

In [ ]:
import os
import cv2
import math
import mediapipe as mp
import csv
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

mp_pose = mp.solutions.pose

def findDistance(x1, y1, x2, y2):
    dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return dist

def findAngle(x1, y1, x2, y2):
    if (x2 - x1) == 0 or y1 == 0:
        return 0
    theta = math.acos((y2 - y1) * (-y1) / (math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) * y1))
    degree = int(180 / math.pi) * theta
    return degree

def process_image(image_path, writer, pose, datagen):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to read image from {image_path}")
        return

    h, w = image.shape[:2]
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Augment images
    augmented_images = []
    augmented_labels = []
    augmented_data = datagen.flow(np.expand_dims(image_rgb, axis=0), batch_size=1)
    for i in range(5):  # Augment each image 5 times
        augmented_image = augmented_data.next()[0].astype(np.uint8)
        augmented_images.append(augmented_image)
        augmented_labels.append('good')

    keypoints = pose.process(image_rgb)
    lm = keypoints.pose_landmarks

    if lm is not None:
        nose_x = lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
        nose_y = lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
        nose_z = lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

        l_shldr_x = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
        l_shldr_y = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
        l_shldr_z = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
        r_shldr_x = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
        r_shldr_y = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
        r_shldr_z=lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
        #offset = findDistance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        #nose_angle = findAngle(l_shldr_x, l_shldr_y, nose_x, nose_y)
        #torso_inclination = findAngle(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        l_shldr_x -= r_shldr_x
        l_shldr_y -= r_shldr_y
        nose_x -= r_shldr_x
        nose_y -= r_shldr_y
        r_shldr_x = 0
        r_shldr_y = 0

        label = 'good'

        # Write original image posture data to CSV file
        writer.writerow([nose_x, nose_y, nose_z, l_shldr_x,l_shldr_y,l_shldr_z,r_shldr_x,r_shldr_y,r_shldr_z,label])

        # Write augmented image posture data to CSV file
        for augmented_image in augmented_images:
            augmented_keypoints = pose.process(augmented_image)
            augmented_lm = augmented_keypoints.pose_landmarks

            if augmented_lm is not None:
                nose_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
                nose_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
                nose_z_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

                l_shldr_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
                l_shldr_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
                l_shldr_z_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
                r_shldr_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
                r_shldr_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
                r_shldr_z_aug=augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h

                writer.writerow([nose_x_aug, nose_y_aug, nose_z_aug, l_shldr_x_aug,l_shldr_y_aug,l_shldr_z_aug,r_shldr_x_aug,r_shldr_y_aug,r_shldr_z_aug,label])
            else:
                print(f"No landmarks detected in augmented image {image_path}")

    else:
        print(f"No landmarks detected in image {image_path}")

def process_images_recursively(folder_path, writer, pose, datagen):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.lower().endswith(".jpg") or filename.lower().endswith(".png"):
                image_path = os.path.join(root, filename)
                print("Processing:", image_path)
                process_image(image_path, writer, pose, datagen)

def main():
    folder_path = '/content/drive/MyDrive/data/training/good'
    csv_file_path = '/content/augoodtr.csv'

    # Define data generator with augmentation parameters
    datagen = ImageDataGenerator(
        rotation_range=20,
        zoom_range=0.2,
        fill_mode='nearest')

    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Nose X', 'Nose Y', 'Nose Z','Left Shoulder X', 'Left Shoulder Y','Left Shoulder Z','Right Shoulder X' ,'Right Shoulder Y', 'Right Shoulder Z','Label'])

        pose = mp_pose.Pose()
        process_images_recursively(folder_path, writer, pose, datagen)
        pose.close()

    print('Finished processing images and saving data to CSV.')

if __name__ == "__main__":
    main()


Processing: /content/drive/MyDrive/data/training/good/good_posture_001.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_002.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_003.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_004.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_005.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_006.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_007.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_008.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_009.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_010.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_011.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_012.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_013.jpg
Processing: /content/driv

In [ ]:
import os
import cv2
import math
import mediapipe as mp
import csv
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

mp_pose = mp.solutions.pose

def findDistance(x1, y1, x2, y2):
    dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return dist

def findAngle(x1, y1, x2, y2):
    if (x2 - x1) == 0 or y1 == 0:
        return 0
    theta = math.acos((y2 - y1) * (-y1) / (math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) * y1))
    degree = int(180 / math.pi) * theta
    return degree

def process_image(image_path, writer, pose, datagen):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to read image from {image_path}")
        return

    h, w = image.shape[:2]
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Augment images
    augmented_images = []
    augmented_labels = []
    augmented_data = datagen.flow(np.expand_dims(image_rgb, axis=0), batch_size=1)
    for i in range(5):  # Augment each image 5 times
        augmented_image = augmented_data.next()[0].astype(np.uint8)
        augmented_images.append(augmented_image)
        augmented_labels.append('bad')

    keypoints = pose.process(image_rgb)
    lm = keypoints.pose_landmarks

    if lm is not None:
        nose_x = lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
        nose_y = lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
        nose_z = lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

        l_shldr_x = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
        l_shldr_y = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
        l_shldr_z = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
        r_shldr_x = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
        r_shldr_y = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
        r_shldr_z=lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
        #offset = findDistance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        #nose_angle = findAngle(l_shldr_x, l_shldr_y, nose_x, nose_y)
        #torso_inclination = findAngle(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        l_shldr_x -= r_shldr_x
        l_shldr_y -= r_shldr_y
        nose_x -= r_shldr_x
        nose_y -= r_shldr_y
        r_shldr_x = 0
        r_shldr_y = 0

        label = 'bad'

        # Write original image posture data to CSV file
        writer.writerow([nose_x, nose_y, nose_z, l_shldr_x,l_shldr_y,l_shldr_z,r_shldr_x,r_shldr_y,r_shldr_z,label])

        # Write augmented image posture data to CSV file
        for augmented_image in augmented_images:
            augmented_keypoints = pose.process(augmented_image)
            augmented_lm = augmented_keypoints.pose_landmarks

            if augmented_lm is not None:
                nose_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
                nose_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
                nose_z_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

                l_shldr_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
                l_shldr_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
                l_shldr_z_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
                r_shldr_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
                r_shldr_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
                r_shldr_z_aug=augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h

                writer.writerow([nose_x_aug, nose_y_aug, nose_z_aug, l_shldr_x_aug,l_shldr_y_aug,l_shldr_z_aug,r_shldr_x_aug,r_shldr_y_aug,r_shldr_z_aug,label])
            else:
                print(f"No landmarks detected in augmented image {image_path}")

    else:
        print(f"No landmarks detected in image {image_path}")

def process_images_recursively(folder_path, writer, pose, datagen):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.lower().endswith(".jpg") or filename.lower().endswith(".png"):
                image_path = os.path.join(root, filename)
                print("Processing:", image_path)
                process_image(image_path, writer, pose, datagen)

def main():
    folder_path = '/content/drive/MyDrive/data/training/bad'
    csv_file_path = '/content/aubadtr.csv'

    # Define data generator with augmentation parameters
    datagen = ImageDataGenerator(
        rotation_range=20,
        zoom_range=0.2,
        fill_mode='nearest')

    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Nose X', 'Nose Y', 'Nose Z','Left Shoulder X', 'Left Shoulder Y','Left Shoulder Z','Right Shoulder X' ,'Right Shoulder Y', 'Right Shoulder Z','Label'])

        pose = mp_pose.Pose()
        process_images_recursively(folder_path, writer, pose, datagen)
        pose.close()

    print('Finished processing images and saving data to CSV.')

if __name__ == "__main__":
    main()


Processing: /content/drive/MyDrive/data/training/bad/bad_posture_001.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_002.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_003.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_004.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_005.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_006.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_007.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_008.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_009.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_010.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_011.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_012.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_013.jpg
Processing: /content/drive/MyDrive/data/training/ba

In [ ]:
import os
import cv2
import math
import mediapipe as mp
import csv
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

mp_pose = mp.solutions.pose

def findDistance(x1, y1, x2, y2):
    dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return dist

def findAngle(x1, y1, x2, y2):
    if (x2 - x1) == 0 or y1 == 0:
        return 0
    theta = math.acos((y2 - y1) * (-y1) / (math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) * y1))
    degree = int(180 / math.pi) * theta
    return degree

def process_image(image_path, writer, pose, datagen):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to read image from {image_path}")
        return

    h, w = image.shape[:2]
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Augment images
    augmented_images = []
    augmented_labels = []
    augmented_data = datagen.flow(np.expand_dims(image_rgb, axis=0), batch_size=1)
    for i in range(5):  # Augment each image 5 times
        augmented_image = augmented_data.next()[0].astype(np.uint8)
        augmented_images.append(augmented_image)
        augmented_labels.append('good')

    keypoints = pose.process(image_rgb)
    lm = keypoints.pose_landmarks

    if lm is not None:
        nose_x = lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
        nose_y = lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
        nose_z = lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

        l_shldr_x = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
        l_shldr_y = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
        l_shldr_z = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
        r_shldr_x = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
        r_shldr_y = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
        r_shldr_z=lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
        #offset = findDistance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        #nose_angle = findAngle(l_shldr_x, l_shldr_y, nose_x, nose_y)
        #torso_inclination = findAngle(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        l_shldr_x -= r_shldr_x
        l_shldr_y -= r_shldr_y
        nose_x -= r_shldr_x
        nose_y -= r_shldr_y
        r_shldr_x = 0
        r_shldr_y = 0

        label = 'good'

        # Write original image posture data to CSV file
        writer.writerow([nose_x, nose_y, nose_z, l_shldr_x,l_shldr_y,l_shldr_z,r_shldr_x,r_shldr_y,r_shldr_z,label])

        # Write augmented image posture data to CSV file
        for augmented_image in augmented_images:
            augmented_keypoints = pose.process(augmented_image)
            augmented_lm = augmented_keypoints.pose_landmarks

            if augmented_lm is not None:
                nose_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
                nose_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
                nose_z_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

                l_shldr_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
                l_shldr_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
                l_shldr_z_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
                r_shldr_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
                r_shldr_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
                r_shldr_z_aug=augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h

                writer.writerow([nose_x_aug, nose_y_aug, nose_z_aug, l_shldr_x_aug,l_shldr_y_aug,l_shldr_z_aug,r_shldr_x_aug,r_shldr_y_aug,r_shldr_z_aug,label])
            else:
                print(f"No landmarks detected in augmented image {image_path}")

    else:
        print(f"No landmarks detected in image {image_path}")

def process_images_recursively(folder_path, writer, pose, datagen):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.lower().endswith(".jpg") or filename.lower().endswith(".png"):
                image_path = os.path.join(root, filename)
                print("Processing:", image_path)
                process_image(image_path, writer, pose, datagen)

def main():
    folder_path = '/content/drive/MyDrive/data/validation/good'
    csv_file_path = '/content/augoodval.csv'

    # Define data generator with augmentation parameters
    datagen = ImageDataGenerator(
        rotation_range=20,
        zoom_range=0.2,
        fill_mode='nearest')

    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Nose X', 'Nose Y', 'Nose Z','Left Shoulder X', 'Left Shoulder Y','Left Shoulder Z','Right Shoulder X' ,'Right Shoulder Y', 'Right Shoulder Z','Label'])

        pose = mp_pose.Pose()
        process_images_recursively(folder_path, writer, pose, datagen)
        pose.close()

    print('Finished processing images and saving data to CSV.')

if __name__ == "__main__":
    main()


Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_001.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_002.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_003.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_004.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_005.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_006.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_007.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_19_12_21_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_19_24_44_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_19_24_46_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_19_24_47_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_19_24_56_Pro.jpg
Pro

In [ ]:
import os
import cv2
import math
import mediapipe as mp
import csv
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

mp_pose = mp.solutions.pose

def findDistance(x1, y1, x2, y2):
    dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return dist

def findAngle(x1, y1, x2, y2):
    if (x2 - x1) == 0 or y1 == 0:
        return 0
    theta = math.acos((y2 - y1) * (-y1) / (math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) * y1))
    degree = int(180 / math.pi) * theta
    return degree

def process_image(image_path, writer, pose, datagen):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to read image from {image_path}")
        return

    h, w = image.shape[:2]
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Augment images
    augmented_images = []
    augmented_labels = []
    augmented_data = datagen.flow(np.expand_dims(image_rgb, axis=0), batch_size=1)
    for i in range(5):  # Augment each image 5 times
        augmented_image = augmented_data.next()[0].astype(np.uint8)
        augmented_images.append(augmented_image)
        augmented_labels.append('bad')

    keypoints = pose.process(image_rgb)
    lm = keypoints.pose_landmarks

    if lm is not None:
        nose_x = lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
        nose_y = lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
        nose_z = lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

        l_shldr_x = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
        l_shldr_y = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
        l_shldr_z = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
        r_shldr_x = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
        r_shldr_y = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
        r_shldr_z=lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
        #offset = findDistance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        #nose_angle = findAngle(l_shldr_x, l_shldr_y, nose_x, nose_y)
        #torso_inclination = findAngle(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        l_shldr_x -= r_shldr_x
        l_shldr_y -= r_shldr_y
        nose_x -= r_shldr_x
        nose_y -= r_shldr_y
        r_shldr_x = 0
        r_shldr_y = 0

        label = 'bad'

        # Write original image posture data to CSV file
        writer.writerow([nose_x, nose_y, nose_z, l_shldr_x,l_shldr_y,l_shldr_z,r_shldr_x,r_shldr_y,r_shldr_z,label])

        # Write augmented image posture data to CSV file
        for augmented_image in augmented_images:
            augmented_keypoints = pose.process(augmented_image)
            augmented_lm = augmented_keypoints.pose_landmarks

            if augmented_lm is not None:
                nose_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
                nose_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
                nose_z_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

                l_shldr_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
                l_shldr_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
                l_shldr_z_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
                r_shldr_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
                r_shldr_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
                r_shldr_z_aug=augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
                l_shldr_x_aug -= r_shldr_x_aug
                l_shldr_y_aug -= r_shldr_y_aug
                nose_x_aug -=r_shldr_x_aug
                nose_y_aug -= r_shldr_y_aug
                r_shldr_x_aug = 0
                r_shldr_y_aug = 0

                writer.writerow([nose_x_aug, nose_y_aug, nose_z_aug, l_shldr_x_aug,l_shldr_y_aug,l_shldr_z_aug,r_shldr_x_aug,r_shldr_y_aug,r_shldr_z_aug,label])
            else:
                print(f"No landmarks detected in augmented image {image_path}")

    else:
        print(f"No landmarks detected in image {image_path}")

def process_images_recursively(folder_path, writer, pose, datagen):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.lower().endswith(".jpg") or filename.lower().endswith(".png"):
                image_path = os.path.join(root, filename)
                print("Processing:", image_path)
                process_image(image_path, writer, pose, datagen)

def main():
    folder_path = '/content/drive/MyDrive/data/training/bad'
    csv_file_path = '/content/au1badtr.csv'

    # Define data generator with augmentation parameters
    datagen = ImageDataGenerator(
        rotation_range=20,
        zoom_range=0.2,
        fill_mode='nearest')

    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Nose X', 'Nose Y', 'Nose Z','Left Shoulder X', 'Left Shoulder Y','Left Shoulder Z','Right Shoulder X' ,'Right Shoulder Y', 'Right Shoulder Z','Label'])

        pose = mp_pose.Pose()
        process_images_recursively(folder_path, writer, pose, datagen)
        pose.close()

    print('Finished processing images and saving data to CSV.')

if __name__ == "__main__":
    main()


Processing: /content/drive/MyDrive/data/training/bad/bad_posture_001.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_002.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_003.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_004.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_005.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_006.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_007.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_008.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_009.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_010.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_011.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_012.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_013.jpg
Processing: /content/drive/MyDrive/data/training/ba

In [ ]:
import os
import cv2
import math
import mediapipe as mp
import csv
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

mp_pose = mp.solutions.pose

def findDistance(x1, y1, x2, y2):
    dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return dist

def findAngle(x1, y1, x2, y2):
    if (x2 - x1) == 0 or y1 == 0:
        return 0
    theta = math.acos((y2 - y1) * (-y1) / (math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) * y1))
    degree = int(180 / math.pi) * theta
    return degree

def process_image(image_path, writer, pose, datagen):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to read image from {image_path}")
        return

    h, w = image.shape[:2]
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Augment images
    augmented_images = []
    augmented_labels = []
    augmented_data = datagen.flow(np.expand_dims(image_rgb, axis=0), batch_size=1)
    for i in range(5):  # Augment each image 5 times
        augmented_image = augmented_data.next()[0].astype(np.uint8)
        augmented_images.append(augmented_image)
        augmented_labels.append('good')

    keypoints = pose.process(image_rgb)
    lm = keypoints.pose_landmarks

    if lm is not None:
        nose_x = lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
        nose_y = lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
        nose_z = lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

        l_shldr_x = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
        l_shldr_y = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
        l_shldr_z = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
        r_shldr_x = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
        r_shldr_y = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
        r_shldr_z=lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
        #offset = findDistance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        #nose_angle = findAngle(l_shldr_x, l_shldr_y, nose_x, nose_y)
        #torso_inclination = findAngle(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        l_shldr_x -= r_shldr_x
        l_shldr_y -= r_shldr_y
        nose_x -= r_shldr_x
        nose_y -= r_shldr_y
        r_shldr_x = 0
        r_shldr_y = 0

        label = 'good'

        # Write original image posture data to CSV file
        writer.writerow([nose_x, nose_y, nose_z, l_shldr_x,l_shldr_y,l_shldr_z,r_shldr_x,r_shldr_y,r_shldr_z,label])

        # Write augmented image posture data to CSV file
        for augmented_image in augmented_images:
            augmented_keypoints = pose.process(augmented_image)
            augmented_lm = augmented_keypoints.pose_landmarks

            if augmented_lm is not None:
                nose_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
                nose_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
                nose_z_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

                l_shldr_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
                l_shldr_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
                l_shldr_z_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
                r_shldr_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
                r_shldr_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
                r_shldr_z_aug=augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
                l_shldr_x_aug -= r_shldr_x_aug
                l_shldr_y_aug -= r_shldr_y_aug
                nose_x_aug -=r_shldr_x_aug
                nose_y_aug -= r_shldr_y_aug
                r_shldr_x_aug = 0
                r_shldr_y_aug = 0

                writer.writerow([nose_x_aug, nose_y_aug, nose_z_aug, l_shldr_x_aug,l_shldr_y_aug,l_shldr_z_aug,r_shldr_x_aug,r_shldr_y_aug,r_shldr_z_aug,label])
            else:
                print(f"No landmarks detected in augmented image {image_path}")

    else:
        print(f"No landmarks detected in image {image_path}")

def process_images_recursively(folder_path, writer, pose, datagen):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.lower().endswith(".jpg") or filename.lower().endswith(".png"):
                image_path = os.path.join(root, filename)
                print("Processing:", image_path)
                process_image(image_path, writer, pose, datagen)

def main():
    folder_path = '/content/drive/MyDrive/data/training/good'
    csv_file_path = '/content/au1goodtr.csv'

    # Define data generator with augmentation parameters
    datagen = ImageDataGenerator(
        rotation_range=20,
        zoom_range=0.2,
        fill_mode='nearest')

    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Nose X', 'Nose Y', 'Nose Z','Left Shoulder X', 'Left Shoulder Y','Left Shoulder Z','Right Shoulder X' ,'Right Shoulder Y', 'Right Shoulder Z','Label'])

        pose = mp_pose.Pose()
        process_images_recursively(folder_path, writer, pose, datagen)
        pose.close()

    print('Finished processing images and saving data to CSV.')

if __name__ == "__main__":
    main()


Processing: /content/drive/MyDrive/data/training/good/good_posture_001.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_002.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_003.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_004.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_005.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_006.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_007.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_008.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_009.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_010.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_011.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_012.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_013.jpg
Processing: /content/driv

In [ ]:
import os
import cv2
import math
import mediapipe as mp
import csv
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

mp_pose = mp.solutions.pose

def findDistance(x1, y1, x2, y2):
    dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return dist

def findAngle(x1, y1, x2, y2):
    if (x2 - x1) == 0 or y1 == 0:
        return 0
    theta = math.acos((y2 - y1) * (-y1) / (math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) * y1))
    degree = int(180 / math.pi) * theta
    return degree

def process_image(image_path, writer, pose, datagen):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to read image from {image_path}")
        return

    h, w = image.shape[:2]
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Augment images
    augmented_images = []
    augmented_labels = []
    augmented_data = datagen.flow(np.expand_dims(image_rgb, axis=0), batch_size=1)
    for i in range(5):  # Augment each image 5 times
        augmented_image = augmented_data.next()[0].astype(np.uint8)
        augmented_images.append(augmented_image)
        augmented_labels.append('good')

    keypoints = pose.process(image_rgb)
    lm = keypoints.pose_landmarks

    if lm is not None:
        nose_x = lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
        nose_y = lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
        nose_z = lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

        l_shldr_x = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
        l_shldr_y = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
        l_shldr_z = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
        r_shldr_x = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
        r_shldr_y = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
        r_shldr_z=lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
        #offset = findDistance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        #nose_angle = findAngle(l_shldr_x, l_shldr_y, nose_x, nose_y)
        #torso_inclination = findAngle(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        l_shldr_x -= r_shldr_x
        l_shldr_y -= r_shldr_y
        nose_x -= r_shldr_x
        nose_y -= r_shldr_y
        r_shldr_x = 0
        r_shldr_y = 0

        label = 'good'

        # Write original image posture data to CSV file
        writer.writerow([nose_x, nose_y, nose_z, l_shldr_x,l_shldr_y,l_shldr_z,r_shldr_x,r_shldr_y,r_shldr_z,label])

        # Write augmented image posture data to CSV file
        for augmented_image in augmented_images:
            augmented_keypoints = pose.process(augmented_image)
            augmented_lm = augmented_keypoints.pose_landmarks

            if augmented_lm is not None:
                nose_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
                nose_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
                nose_z_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

                l_shldr_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
                l_shldr_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
                l_shldr_z_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
                r_shldr_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
                r_shldr_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
                r_shldr_z_aug=augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
                l_shldr_x_aug -= r_shldr_x_aug
                l_shldr_y_aug -= r_shldr_y_aug
                nose_x_aug -=r_shldr_x_aug
                nose_y_aug -= r_shldr_y_aug
                r_shldr_x_aug = 0
                r_shldr_y_aug = 0

                writer.writerow([nose_x_aug, nose_y_aug, nose_z_aug, l_shldr_x_aug,l_shldr_y_aug,l_shldr_z_aug,r_shldr_x_aug,r_shldr_y_aug,r_shldr_z_aug,label])
            else:
                print(f"No landmarks detected in augmented image {image_path}")

    else:
        print(f"No landmarks detected in image {image_path}")

def process_images_recursively(folder_path, writer, pose, datagen):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.lower().endswith(".jpg") or filename.lower().endswith(".png"):
                image_path = os.path.join(root, filename)
                print("Processing:", image_path)
                process_image(image_path, writer, pose, datagen)

def main():
    folder_path = '/content/drive/MyDrive/data/validation/good'
    csv_file_path = '/content/au1goodval.csv'

    # Define data generator with augmentation parameters
    datagen = ImageDataGenerator(
        rotation_range=20,
        zoom_range=0.2,
        fill_mode='nearest')

    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Nose X', 'Nose Y', 'Nose Z','Left Shoulder X', 'Left Shoulder Y','Left Shoulder Z','Right Shoulder X' ,'Right Shoulder Y', 'Right Shoulder Z','Label'])

        pose = mp_pose.Pose()
        process_images_recursively(folder_path, writer, pose, datagen)
        pose.close()

    print('Finished processing images and saving data to CSV.')

if __name__ == "__main__":
    main()


Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_001.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_002.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_003.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_004.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_005.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_006.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_007.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_19_12_21_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_19_24_44_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_19_24_46_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_19_24_47_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_19_24_56_Pro.jpg
Pro

In [ ]:
import os
import cv2
import math
import mediapipe as mp
import csv
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

mp_pose = mp.solutions.pose

def findDistance(x1, y1, x2, y2):
    dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return dist

def findAngle(x1, y1, x2, y2):
    if (x2 - x1) == 0 or y1 == 0:
        return 0
    theta = math.acos((y2 - y1) * (-y1) / (math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) * y1))
    degree = int(180 / math.pi) * theta
    return degree

def process_image(image_path, writer, pose, datagen):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to read image from {image_path}")
        return

    h, w = image.shape[:2]
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Augment images
    augmented_images = []
    augmented_labels = []
    augmented_data = datagen.flow(np.expand_dims(image_rgb, axis=0), batch_size=1)
    for i in range(5):  # Augment each image 5 times
        augmented_image = augmented_data.next()[0].astype(np.uint8)
        augmented_images.append(augmented_image)
        augmented_labels.append('bad')

    keypoints = pose.process(image_rgb)
    lm = keypoints.pose_landmarks

    if lm is not None:
        nose_x = lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
        nose_y = lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
        nose_z = lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

        l_shldr_x = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
        l_shldr_y = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
        l_shldr_z = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
        r_shldr_x = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
        r_shldr_y = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
        r_shldr_z=lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
        #offset = findDistance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        #nose_angle = findAngle(l_shldr_x, l_shldr_y, nose_x, nose_y)
        #torso_inclination = findAngle(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        l_shldr_x -= r_shldr_x
        l_shldr_y -= r_shldr_y
        nose_x -= r_shldr_x
        nose_y -= r_shldr_y
        r_shldr_x = 0
        r_shldr_y = 0

        label = 'bad'

        # Write original image posture data to CSV file
        writer.writerow([nose_x, nose_y, nose_z, l_shldr_x,l_shldr_y,l_shldr_z,r_shldr_x,r_shldr_y,r_shldr_z,label])

        # Write augmented image posture data to CSV file
        for augmented_image in augmented_images:
            augmented_keypoints = pose.process(augmented_image)
            augmented_lm = augmented_keypoints.pose_landmarks

            if augmented_lm is not None:
                nose_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
                nose_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
                nose_z_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

                l_shldr_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
                l_shldr_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
                l_shldr_z_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
                r_shldr_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
                r_shldr_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
                r_shldr_z_aug=augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
                l_shldr_x_aug -= r_shldr_x_aug
                l_shldr_y_aug -= r_shldr_y_aug
                nose_x_aug -=r_shldr_x_aug
                nose_y_aug -= r_shldr_y_aug
                r_shldr_x_aug = 0
                r_shldr_y_aug = 0

                writer.writerow([nose_x_aug, nose_y_aug, nose_z_aug, l_shldr_x_aug,l_shldr_y_aug,l_shldr_z_aug,r_shldr_x_aug,r_shldr_y_aug,r_shldr_z_aug,label])
            else:
                print(f"No landmarks detected in augmented image {image_path}")

    else:
        print(f"No landmarks detected in image {image_path}")

def process_images_recursively(folder_path, writer, pose, datagen):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.lower().endswith(".jpg") or filename.lower().endswith(".png"):
                image_path = os.path.join(root, filename)
                print("Processing:", image_path)
                process_image(image_path, writer, pose, datagen)

def main():
    folder_path = '/content/drive/MyDrive/data/training/good'
    csv_file_path = '/content/newbad.csv'

    # Define data generator with augmentation parameters
    datagen = ImageDataGenerator(
        rotation_range=40,
        zoom_range=0.2,
        fill_mode='nearest')

    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Nose X', 'Nose Y', 'Nose Z','Left Shoulder X', 'Left Shoulder Y','Left Shoulder Z','Right Shoulder X' ,'Right Shoulder Y', 'Right Shoulder Z','Label'])

        pose = mp_pose.Pose()
        process_images_recursively(folder_path, writer, pose, datagen)
        pose.close()

    print('Finished processing images and saving data to CSV.')

if __name__ == "__main__":
    main()


Processing: /content/drive/MyDrive/data/training/good/good_posture_001.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_002.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_003.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_004.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_005.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_006.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_007.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_008.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_009.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_010.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_011.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_012.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_013.jpg
Processing: /content/driv

In [ ]:
import os
import cv2
import math
import mediapipe as mp
import csv
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

mp_pose = mp.solutions.pose

def findDistance(x1, y1, x2, y2):
    dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return dist

def findAngle(x1, y1, x2, y2):
    if (x2 - x1) == 0 or y1 == 0:
        return 0
    theta = math.acos((y2 - y1) * (-y1) / (math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) * y1))
    degree = int(180 / math.pi) * theta
    return degree

def process_image(image_path, writer, pose, datagen):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to read image from {image_path}")
        return

    h, w = image.shape[:2]
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Augment images
    augmented_images = []
    augmented_labels = []
    augmented_data = datagen.flow(np.expand_dims(image_rgb, axis=0), batch_size=1)
    for i in range(5):  # Augment each image 5 times
        augmented_image = augmented_data.next()[0].astype(np.uint8)
        augmented_images.append(augmented_image)
        augmented_labels.append('bad')

    keypoints = pose.process(image_rgb)
    lm = keypoints.pose_landmarks

    if lm is not None:
        nose_x = lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
        nose_y = lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
        nose_z = lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

        l_shldr_x = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
        l_shldr_y = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
        l_shldr_z = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
        r_shldr_x = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
        r_shldr_y = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
        r_shldr_z=lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
        #offset = findDistance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        #nose_angle = findAngle(l_shldr_x, l_shldr_y, nose_x, nose_y)
        #torso_inclination = findAngle(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        l_shldr_x -= r_shldr_x
        l_shldr_y -= r_shldr_y
        nose_x -= r_shldr_x
        nose_y -= r_shldr_y
        r_shldr_x = 0
        r_shldr_y = 0

        label = 'bad'

        # Write original image posture data to CSV file
        writer.writerow([nose_x, nose_y, nose_z, l_shldr_x,l_shldr_y,l_shldr_z,r_shldr_x,r_shldr_y,r_shldr_z,label])

        # Write augmented image posture data to CSV file
        for augmented_image in augmented_images:
            augmented_keypoints = pose.process(augmented_image)
            augmented_lm = augmented_keypoints.pose_landmarks

            if augmented_lm is not None:
                nose_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
                nose_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
                nose_z_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

                l_shldr_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
                l_shldr_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
                l_shldr_z_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
                r_shldr_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
                r_shldr_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
                r_shldr_z_aug=augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
                l_shldr_x_aug -= r_shldr_x_aug
                l_shldr_y_aug -= r_shldr_y_aug
                nose_x_aug -=r_shldr_x_aug
                nose_y_aug -= r_shldr_y_aug
                r_shldr_x_aug = 0
                r_shldr_y_aug = 0

                writer.writerow([nose_x_aug, nose_y_aug, nose_z_aug, l_shldr_x_aug,l_shldr_y_aug,l_shldr_z_aug,r_shldr_x_aug,r_shldr_y_aug,r_shldr_z_aug,label])
            else:
                print(f"No landmarks detected in augmented image {image_path}")

    else:
        print(f"No landmarks detected in image {image_path}")

def process_images_recursively(folder_path, writer, pose, datagen):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.lower().endswith(".jpg") or filename.lower().endswith(".png"):
                image_path = os.path.join(root, filename)
                print("Processing:", image_path)
                process_image(image_path, writer, pose, datagen)

def main():
    folder_path = '/content/drive/MyDrive/data/training/bad'
    csv_file_path = '/content/newbadtrb.csv'

    # Define data generator with augmentation parameters
    datagen = ImageDataGenerator(
        rotation_range=40,
        zoom_range=0.2,
        fill_mode='nearest')

    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Nose X', 'Nose Y', 'Nose Z','Left Shoulder X', 'Left Shoulder Y','Left Shoulder Z','Right Shoulder X' ,'Right Shoulder Y', 'Right Shoulder Z','Label'])

        pose = mp_pose.Pose()
        process_images_recursively(folder_path, writer, pose, datagen)
        pose.close()

    print('Finished processing images and saving data to CSV.')

if __name__ == "__main__":
    main()


Processing: /content/drive/MyDrive/data/training/bad/bad_posture_001.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_002.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_003.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_004.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_005.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_006.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_007.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_008.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_009.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_010.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_011.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_012.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_013.jpg
Processing: /content/drive/MyDrive/data/training/ba

In [ ]:
import os
import cv2
import math
import mediapipe as mp
import csv
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

mp_pose = mp.solutions.pose

def findDistance(x1, y1, x2, y2):
    dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return dist

def findAngle(x1, y1, x2, y2):
    if (x2 - x1) == 0 or y1 == 0:
        return 0
    theta = math.acos((y2 - y1) * (-y1) / (math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) * y1))
    degree = int(180 / math.pi) * theta
    return degree

def process_image(image_path, writer, pose, datagen):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to read image from {image_path}")
        return

    h, w = image.shape[:2]
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Augment images
    augmented_images = []
    augmented_labels = []
    augmented_data = datagen.flow(np.expand_dims(image_rgb, axis=0), batch_size=1)
    for i in range(5):  # Augment each image 5 times
        augmented_image = augmented_data.next()[0].astype(np.uint8)
        augmented_images.append(augmented_image)
        augmented_labels.append('bad')

    keypoints = pose.process(image_rgb)
    lm = keypoints.pose_landmarks

    if lm is not None:
        nose_x = lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
        nose_y = lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
        nose_z = lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

        l_shldr_x = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
        l_shldr_y = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
        l_shldr_z = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
        r_shldr_x = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
        r_shldr_y = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
        r_shldr_z=lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
        #offset = findDistance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        #nose_angle = findAngle(l_shldr_x, l_shldr_y, nose_x, nose_y)
        #torso_inclination = findAngle(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        l_shldr_x -= r_shldr_x
        l_shldr_y -= r_shldr_y
        nose_x -= r_shldr_x
        nose_y -= r_shldr_y
        r_shldr_x = 0
        r_shldr_y = 0

        label = 'bad'

        # Write original image posture data to CSV file
        writer.writerow([nose_x, nose_y, nose_z, l_shldr_x,l_shldr_y,l_shldr_z,r_shldr_x,r_shldr_y,r_shldr_z,label])

        # Write augmented image posture data to CSV file
        for augmented_image in augmented_images:
            augmented_keypoints = pose.process(augmented_image)
            augmented_lm = augmented_keypoints.pose_landmarks

            if augmented_lm is not None:
                nose_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
                nose_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
                nose_z_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

                l_shldr_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
                l_shldr_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
                l_shldr_z_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
                r_shldr_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
                r_shldr_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
                r_shldr_z_aug=augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
                l_shldr_x_aug -= r_shldr_x_aug
                l_shldr_y_aug -= r_shldr_y_aug
                nose_x_aug -=r_shldr_x_aug
                nose_y_aug -= r_shldr_y_aug
                r_shldr_x_aug = 0
                r_shldr_y_aug = 0

                writer.writerow([nose_x_aug, nose_y_aug, nose_z_aug, l_shldr_x_aug,l_shldr_y_aug,l_shldr_z_aug,r_shldr_x_aug,r_shldr_y_aug,r_shldr_z_aug,label])
            else:
                print(f"No landmarks detected in augmented image {image_path}")

    else:
        print(f"No landmarks detected in image {image_path}")

def process_images_recursively(folder_path, writer, pose, datagen):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.lower().endswith(".jpg") or filename.lower().endswith(".png"):
                image_path = os.path.join(root, filename)
                print("Processing:", image_path)
                process_image(image_path, writer, pose, datagen)

def main():
    folder_path = '/content/drive/MyDrive/data/validation/bad'
    csv_file_path = '/content/newbadvalb.csv'

    # Define data generator with augmentation parameters
    datagen = ImageDataGenerator(
        rotation_range=40,
        zoom_range=0.2,
        fill_mode='nearest')

    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Nose X', 'Nose Y', 'Nose Z','Left Shoulder X', 'Left Shoulder Y','Left Shoulder Z','Right Shoulder X' ,'Right Shoulder Y', 'Right Shoulder Z','Label'])

        pose = mp_pose.Pose()
        process_images_recursively(folder_path, writer, pose, datagen)
        pose.close()

    print('Finished processing images and saving data to CSV.')

if __name__ == "__main__":
    main()


Processing: /content/drive/MyDrive/data/validation/bad/bad_posture_val_001.jpg
Processing: /content/drive/MyDrive/data/validation/bad/bad_posture_val_002.jpg
Processing: /content/drive/MyDrive/data/validation/bad/bad_posture_val_003.jpg
Processing: /content/drive/MyDrive/data/validation/bad/bad_posture_val_004.jpg
Processing: /content/drive/MyDrive/data/validation/bad/bad_posture_val_005.jpg
Processing: /content/drive/MyDrive/data/validation/bad/bad_posture_val_006.jpg
Processing: /content/drive/MyDrive/data/validation/bad/bad_posture_val_007.jpg
Processing: /content/drive/MyDrive/data/validation/bad/bad_posture_val_008.jpg
Processing: /content/drive/MyDrive/data/validation/bad/WIN_20240429_19_24_52_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/bad/WIN_20240429_19_24_53_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/bad/WIN_20240429_19_24_54_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/bad/WIN_20240429_19_25_36_Pro.jpg
Processing: /content/drive/M

In [ ]:
import os
import cv2
import math
import mediapipe as mp
import csv
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

mp_pose = mp.solutions.pose

def findDistance(x1, y1, x2, y2):
    dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return dist

def findAngle(x1, y1, x2, y2):
    if (x2 - x1) == 0 or y1 == 0:
        return 0
    theta = math.acos((y2 - y1) * (-y1) / (math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) * y1))
    degree = int(180 / math.pi) * theta
    return degree

def process_image(image_path, writer, pose, datagen):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to read image from {image_path}")
        return

    h, w = image.shape[:2]
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Augment images
    augmented_images = []
    augmented_labels = []
    augmented_data = datagen.flow(np.expand_dims(image_rgb, axis=0), batch_size=1)
    for i in range(5):  # Augment each image 5 times
        augmented_image = augmented_data.next()[0].astype(np.uint8)
        augmented_images.append(augmented_image)
        augmented_labels.append('bad')

    keypoints = pose.process(image_rgb)
    lm = keypoints.pose_landmarks

    if lm is not None:
        nose_x = lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
        nose_y = lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
        nose_z = lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

        l_shldr_x = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
        l_shldr_y = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
        l_shldr_z = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
        r_shldr_x = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
        r_shldr_y = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
        r_shldr_z=lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
        #offset = findDistance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        #nose_angle = findAngle(l_shldr_x, l_shldr_y, nose_x, nose_y)
        #torso_inclination = findAngle(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        l_shldr_x -= r_shldr_x
        l_shldr_y -= r_shldr_y
        nose_x -= r_shldr_x
        nose_y -= r_shldr_y
        r_shldr_x = 0
        r_shldr_y = 0

        label = 'bad'

        # Write original image posture data to CSV file
        writer.writerow([nose_x, nose_y, nose_z, l_shldr_x,l_shldr_y,l_shldr_z,r_shldr_x,r_shldr_y,r_shldr_z,label])

        # Write augmented image posture data to CSV file
        for augmented_image in augmented_images:
            augmented_keypoints = pose.process(augmented_image)
            augmented_lm = augmented_keypoints.pose_landmarks

            if augmented_lm is not None:
                nose_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
                nose_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
                nose_z_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

                l_shldr_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
                l_shldr_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
                l_shldr_z_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
                r_shldr_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
                r_shldr_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
                r_shldr_z_aug=augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
                l_shldr_x_aug -= r_shldr_x_aug
                l_shldr_y_aug -= r_shldr_y_aug
                nose_x_aug -=r_shldr_x_aug
                nose_y_aug -= r_shldr_y_aug
                r_shldr_x_aug = 0
                r_shldr_y_aug = 0

                writer.writerow([nose_x_aug, nose_y_aug, nose_z_aug, l_shldr_x_aug,l_shldr_y_aug,l_shldr_z_aug,r_shldr_x_aug,r_shldr_y_aug,r_shldr_z_aug,label])
            else:
                print(f"No landmarks detected in augmented image {image_path}")

    else:
        print(f"No landmarks detected in image {image_path}")

def process_images_recursively(folder_path, writer, pose, datagen):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.lower().endswith(".jpg") or filename.lower().endswith(".png"):
                image_path = os.path.join(root, filename)
                print("Processing:", image_path)
                process_image(image_path, writer, pose, datagen)

def main():
    folder_path = '/content/drive/MyDrive/data/validation/good'
    csv_file_path = '/content/newbadvalg.csv'

    # Define data generator with augmentation parameters
    datagen = ImageDataGenerator(
        rotation_range=40,
        zoom_range=0.2,
        fill_mode='nearest')

    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Nose X', 'Nose Y', 'Nose Z','Left Shoulder X', 'Left Shoulder Y','Left Shoulder Z','Right Shoulder X' ,'Right Shoulder Y', 'Right Shoulder Z','Label'])

        pose = mp_pose.Pose()
        process_images_recursively(folder_path, writer, pose, datagen)
        pose.close()

    print('Finished processing images and saving data to CSV.')

if __name__ == "__main__":
    main()


Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_001.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_002.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_003.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_004.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_005.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_006.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_007.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_19_12_21_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_19_24_44_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_19_24_46_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_19_24_47_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_19_24_56_Pro.jpg
Pro

In [ ]:
import os
import cv2
import math
import mediapipe as mp
import csv
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

mp_pose = mp.solutions.pose

def findDistance(x1, y1, x2, y2):
    dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return dist

def findAngle(x1, y1, x2, y2):
    if (x2 - x1) == 0 or y1 == 0:
        return 0
    theta = math.acos((y2 - y1) * (-y1) / (math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) * y1))
    degree = int(180 / math.pi) * theta
    return degree

def process_image(image_path, writer, pose, datagen):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to read image from {image_path}")
        return

    h, w = image.shape[:2]
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Augment images
    augmented_images = []
    augmented_labels = []
    augmented_data = datagen.flow(np.expand_dims(image_rgb, axis=0), batch_size=1)
    for i in range(5):  # Augment each image 5 times
        augmented_image = augmented_data.next()[0].astype(np.uint8)
        augmented_images.append(augmented_image)
        augmented_labels.append('good')

    keypoints = pose.process(image_rgb)
    lm = keypoints.pose_landmarks

    if lm is not None:
        nose_x = lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
        nose_y = lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
        nose_z = lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

        l_shldr_x = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
        l_shldr_y = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
        l_shldr_z = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
        r_shldr_x = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
        r_shldr_y = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
        r_shldr_z=lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
        #offset = findDistance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        #nose_angle = findAngle(l_shldr_x, l_shldr_y, nose_x, nose_y)
        #torso_inclination = findAngle(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
        l_shldr_x -= r_shldr_x
        l_shldr_y -= r_shldr_y
        nose_x -= r_shldr_x
        nose_y -= r_shldr_y
        r_shldr_x = 0
        r_shldr_y = 0

        label = 'good'

        # Write original image posture data to CSV file
        writer.writerow([nose_x, nose_y, nose_z, l_shldr_x,l_shldr_y,l_shldr_z,r_shldr_x,r_shldr_y,r_shldr_z,label])

        # Write augmented image posture data to CSV file
        for augmented_image in augmented_images:
            augmented_keypoints = pose.process(augmented_image)
            augmented_lm = augmented_keypoints.pose_landmarks

            if augmented_lm is not None:
                nose_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
                nose_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
                nose_z_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

                l_shldr_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
                l_shldr_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
                l_shldr_z_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
                r_shldr_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
                r_shldr_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
                r_shldr_z_aug=augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
                l_shldr_x_aug -= r_shldr_x_aug
                l_shldr_y_aug -= r_shldr_y_aug
                nose_x_aug -=r_shldr_x_aug
                nose_y_aug -= r_shldr_y_aug
                r_shldr_x_aug = 0
                r_shldr_y_aug = 0

                writer.writerow([nose_x_aug, nose_y_aug, nose_z_aug, l_shldr_x_aug,l_shldr_y_aug,l_shldr_z_aug,r_shldr_x_aug,r_shldr_y_aug,r_shldr_z_aug,label])
            else:
                print(f"No landmarks detected in augmented image {image_path}")

    else:
        print(f"No landmarks detected in image {image_path}")

def process_images_recursively(folder_path, writer, pose, datagen):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.lower().endswith(".jpg") or filename.lower().endswith(".png"):
                image_path = os.path.join(root, filename)
                print("Processing:", image_path)
                process_image(image_path, writer, pose, datagen)

def main():
    folder_path = '/content/drive/MyDrive/data/training/good'
    csv_file_path = '/content/newgoodtr.csv'

    # Define data generator with augmentation parameters
    datagen = ImageDataGenerator(
        zoom_range=0.2,
        fill_mode='nearest')

    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Nose X', 'Nose Y', 'Nose Z','Left Shoulder X', 'Left Shoulder Y','Left Shoulder Z','Right Shoulder X' ,'Right Shoulder Y', 'Right Shoulder Z','Label'])

        pose = mp_pose.Pose()
        process_images_recursively(folder_path, writer, pose, datagen)
        pose.close()

    print('Finished processing images and saving data to CSV.')

if __name__ == "__main__":
    main()


Processing: /content/drive/MyDrive/data/training/good/good_posture_001.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_002.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_003.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_004.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_005.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_006.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_007.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_008.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_009.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_010.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_011.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_012.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_013.jpg
Processing: /content/driv

In [ ]:
import os
import cv2
import math
import mediapipe as mp
import csv
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

mp_pose = mp.solutions.pose

def findDistance(x1, y1, x2, y2):
    dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return dist

def findAngle(x1, y1, x2, y2):
    if (x2 - x1) == 0 or y1 == 0:
        return 0
    theta = math.acos((y2 - y1) * (-y1) / (math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) * y1))
    degree = int(180 / math.pi) * theta
    return degree

def process_image(image_path, writer, pose, datagen):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to read image from {image_path}")
        return

    h, w = image.shape[:2]
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Augment images
    augmented_images = []
    augmented_labels = []
    augmented_data = datagen.flow(np.expand_dims(image_rgb, axis=0), batch_size=1)
    for i in range(5):  # Augment each image 5 times
        augmented_image = augmented_data.next()[0].astype(np.uint8)
        augmented_images.append(augmented_image)
        augmented_labels.append('bad')

    keypoints = pose.process(image_rgb)
    lm = keypoints.pose_landmarks

    if lm is not None:
        nose_x = lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
        nose_y = lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
        nose_z = lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

        l_shldr_x = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
        l_shldr_y = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
        l_shldr_z = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
        r_shldr_x = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
        r_shldr_y = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
        r_shldr_z=lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
        l_shldr_x -= r_shldr_x
        l_shldr_y -= r_shldr_y
        nose_x -= r_shldr_x
        nose_y -= r_shldr_y
        r_shldr_x = 0
        r_shldr_y = 0

        label = 'bad'

        # Write original image posture data to CSV file
        writer.writerow([nose_x, nose_y, nose_z, l_shldr_x,l_shldr_y,l_shldr_z,r_shldr_x,r_shldr_y,r_shldr_z,label])

        # Write augmented image posture data to CSV file
        for augmented_image in augmented_images:
            augmented_keypoints = pose.process(augmented_image)
            augmented_lm = augmented_keypoints.pose_landmarks

            if augmented_lm is not None:
                nose_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
                nose_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
                nose_z_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

                l_shldr_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
                l_shldr_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
                l_shldr_z_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
                r_shldr_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
                r_shldr_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
                r_shldr_z_aug=augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
                l_shldr_x_aug -= r_shldr_x_aug
                l_shldr_y_aug -= r_shldr_y_aug
                nose_x_aug -=r_shldr_x_aug
                nose_y_aug -= r_shldr_y_aug
                r_shldr_x_aug = 0
                r_shldr_y_aug = 0

                writer.writerow([nose_x_aug, nose_y_aug, nose_z_aug, l_shldr_x_aug,l_shldr_y_aug,l_shldr_z_aug,r_shldr_x_aug,r_shldr_y_aug,r_shldr_z_aug,label])
            else:
                print(f"No landmarks detected in augmented image {image_path}")

    else:
        print(f"No landmarks detected in image {image_path}")

def process_images_recursively(folder_path, writer, pose, datagen):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.lower().endswith(".jpg") or filename.lower().endswith(".png"):
                image_path = os.path.join(root, filename)
                print("Processing:", image_path)
                process_image(image_path, writer, pose, datagen)

def main():
    folder_path = '/content/drive/MyDrive/data/training/good'
    csv_file_path = '/content/finbadtrg.csv'

    # Define data generator with augmentation parameters
    datagen = ImageDataGenerator(
        rotation_range=40,
        zoom_range=0.2,
        fill_mode='nearest')

    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Nose X', 'Nose Y', 'Nose Z','Left Shoulder X', 'Left Shoulder Y','Left Shoulder Z','Right Shoulder X' ,'Right Shoulder Y', 'Right Shoulder Z','Label'])

        pose = mp_pose.Pose()
        process_images_recursively(folder_path, writer, pose, datagen)
        pose.close()

    print('Finished processing images and saving data to CSV.')

if __name__ == "__main__":
    main()


Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_001.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_002.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_003.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_004.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_005.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_006.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_007.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_19_12_21_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_19_24_44_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_19_24_46_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_19_24_47_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_19_24_56_Pro.jpg
Pro

In [ ]:
import os
import cv2
import math
import mediapipe as mp
import csv
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

mp_pose = mp.solutions.pose

def findDistance(x1, y1, x2, y2):
    dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return dist

def findAngle(x1, y1, x2, y2):
    if (x2 - x1) == 0 or y1 == 0:
        return 0
    theta = math.acos((y2 - y1) * (-y1) / (math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) * y1))
    degree = int(180 / math.pi) * theta
    return degree

def process_image(image_path, writer, pose, datagen):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to read image from {image_path}")
        return

    h, w = image.shape[:2]
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Augment images
    augmented_images = []
    augmented_labels = []
    augmented_data = datagen.flow(np.expand_dims(image_rgb, axis=0), batch_size=1)
    for i in range(5):  # Augment each image 5 times
        augmented_image = augmented_data.next()[0].astype(np.uint8)
        augmented_images.append(augmented_image)
        augmented_labels.append('bad')

    keypoints = pose.process(image_rgb)
    lm = keypoints.pose_landmarks

    if lm is not None:
        nose_x = lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
        nose_y = lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
        nose_z = lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

        l_shldr_x = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
        l_shldr_y = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
        l_shldr_z = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
        r_shldr_x = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
        r_shldr_y = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
        r_shldr_z=lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
        l_shldr_x -= r_shldr_x
        l_shldr_y -= r_shldr_y
        nose_x -= r_shldr_x
        nose_y -= r_shldr_y

        label = 'bad'
        left_eye_x = lm.landmark[mp_pose.PoseLandmark.LEFT_EYE].x * w - r_shldr_x
        left_eye_y = lm.landmark[mp_pose.PoseLandmark.LEFT_EYE].y * h - r_shldr_y
        left_eye_z = lm.landmark[mp_pose.PoseLandmark.LEFT_EYE].z * h
        right_eye_x = lm.landmark[mp_pose.PoseLandmark.RIGHT_EYE].x * w - r_shldr_x
        right_eye_y = lm.landmark[mp_pose.PoseLandmark.RIGHT_EYE].y * h - r_shldr_y
        right_eye_z = lm.landmark[mp_pose.PoseLandmark.RIGHT_EYE].z * h
        left_lip_x = lm.landmark[mp_pose.PoseLandmark.MOUTH_LEFT].x * w - r_shldr_x
        left_lip_y = lm.landmark[mp_pose.PoseLandmark.MOUTH_LEFT].y * h - r_shldr_y
        left_lip_z = lm.landmark[mp_pose.PoseLandmark.MOUTH_LEFT].z * h
        right_lip_x = lm.landmark[mp_pose.PoseLandmark.MOUTH_RIGHT].x * w - r_shldr_x
        right_lip_y = lm.landmark[mp_pose.PoseLandmark.MOUTH_RIGHT].y * h - r_shldr_y
        right_lip_z = lm.landmark[mp_pose.PoseLandmark.MOUTH_RIGHT].z * h
        r_shldr_x = 0
        r_shldr_y = 0

        writer.writerow([nose_x, nose_y, nose_z, l_shldr_x,l_shldr_y,l_shldr_z,r_shldr_x,r_shldr_y,r_shldr_z,left_eye_x, left_eye_y,left_eye_z, right_eye_x, right_eye_y,right_eye_z ,left_lip_x, left_lip_y, left_lip_z ,right_lip_x, right_lip_y,right_lip_z,label])

        # Write augmented image posture data to CSV file
        for augmented_image in augmented_images:
            augmented_keypoints = pose.process(augmented_image)
            augmented_lm = augmented_keypoints.pose_landmarks

            if augmented_lm is not None:
                nose_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
                nose_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
                nose_z_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

                l_shldr_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
                l_shldr_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
                l_shldr_z_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
                r_shldr_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
                r_shldr_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
                r_shldr_z_aug=augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
                l_shldr_x_aug -= r_shldr_x_aug
                l_shldr_y_aug -= r_shldr_y_aug
                nose_x_aug -=r_shldr_x_aug
                nose_y_aug -= r_shldr_y_aug

                left_eye_x_aug=augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_EYE].x * w- r_shldr_x_aug
                left_eye_y_aug=augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_EYE].y * h- r_shldr_y_aug
                left_eye_z_aug=augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_EYE].z * h
                right_eye_x_aug=augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_EYE].x * w- r_shldr_x_aug
                right_eye_y_aug=augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_EYE].y * h- r_shldr_y_aug
                right_eye_z_aug=augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_EYE].z * h
                left_lip_x_aug=augmented_lm.landmark[mp_pose.PoseLandmark.MOUTH_LEFT].x * w- r_shldr_x_aug
                left_lip_y_aug=augmented_lm.landmark[mp_pose.PoseLandmark.MOUTH_LEFT].y * h- r_shldr_y_aug
                left_lip_z_aug=augmented_lm.landmark[mp_pose.PoseLandmark.MOUTH_LEFT].z * h
                right_lip_x_aug=augmented_lm.landmark[mp_pose.PoseLandmark.MOUTH_RIGHT].x * w- r_shldr_x_aug
                right_lip_y_aug=augmented_lm.landmark[mp_pose.PoseLandmark.MOUTH_RIGHT].y * h- r_shldr_y_aug
                right_lip_z_aug=augmented_lm.landmark[mp_pose.PoseLandmark.MOUTH_RIGHT].z * h

                r_shldr_x_aug = 0
                r_shldr_y_aug = 0

                writer.writerow([nose_x_aug, nose_y_aug, nose_z_aug, l_shldr_x_aug,l_shldr_y_aug,l_shldr_z_aug,r_shldr_x_aug,r_shldr_y_aug,r_shldr_z_aug,left_eye_x,left_eye_y,left_eye_z,right_eye_x,right_eye_y,right_eye_z,left_lip_x,left_lip_y,left_lip_z,right_lip_x,right_lip_y,right_lip_z,label])
            else:
                print(f"No landmarks detected in augmented image {image_path}")

    else:
        print(f"No landmarks detected in image {image_path}")

def process_images_recursively(folder_path, writer, pose, datagen):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.lower().endswith(".jpg") or filename.lower().endswith(".png"):
                image_path = os.path.join(root, filename)
                print("Processing:", image_path)
                process_image(image_path, writer, pose, datagen)

def main():
    folder_path = '/content/drive/MyDrive/data/training/good'
    csv_file_path = '/content/finbadtrg.csv'

    # Define data generator with augmentation parameters
    datagen = ImageDataGenerator(
        zoom_range=0.2,
        fill_mode='nearest')

    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Nose X', 'Nose Y', 'Nose Z','Left Shoulder X', 'Left Shoulder Y','Left Shoulder Z','Right Shoulder X' ,'Right Shoulder Y', 'Right Shoulder Z','Left Eye X','Left Eye Y','Left Eye Z','Right Eye X','Right Eye Y','Right Eye Z','Left Lip X','Left Lip Y','Left Lip Z','Right Lip X','Right Lip Y','Right Lip Z','Label'])

        pose = mp_pose.Pose()
        process_images_recursively(folder_path, writer, pose, datagen)
        pose.close()

    print('Finished processing images and saving data to CSV.')

if __name__ == "__main__":
    main()


Processing: /content/drive/MyDrive/data/training/good/good_posture_001.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_002.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_003.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_004.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_005.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_006.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_007.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_008.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_009.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_010.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_011.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_012.jpg
Processing: /content/drive/MyDrive/data/training/good/good_posture_013.jpg
Processing: /content/driv

In [ ]:
import os
import cv2
import math
import mediapipe as mp
import csv
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

mp_pose = mp.solutions.pose

def findDistance(x1, y1, x2, y2):
    dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return dist

def findAngle(x1, y1, x2, y2):
    if (x2 - x1) == 0 or y1 == 0:
        return 0
    theta = math.acos((y2 - y1) * (-y1) / (math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) * y1))
    degree = int(180 / math.pi) * theta
    return degree

def process_image(image_path, writer, pose, datagen):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to read image from {image_path}")
        return

    h, w = image.shape[:2]
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Augment images
    augmented_images = []
    augmented_labels = []
    augmented_data = datagen.flow(np.expand_dims(image_rgb, axis=0), batch_size=1)
    for i in range(5):  # Augment each image 5 times
        augmented_image = augmented_data.next()[0].astype(np.uint8)
        augmented_images.append(augmented_image)
        augmented_labels.append('bad')

    keypoints = pose.process(image_rgb)
    lm = keypoints.pose_landmarks

    if lm is not None:
        nose_x = lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
        nose_y = lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
        nose_z = lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

        l_shldr_x = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
        l_shldr_y = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
        l_shldr_z = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
        r_shldr_x = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
        r_shldr_y = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
        r_shldr_z=lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
        l_shldr_x -= r_shldr_x
        l_shldr_y -= r_shldr_y
        nose_x -= r_shldr_x
        nose_y -= r_shldr_y

        label = 'bad'
        left_eye_x = lm.landmark[mp_pose.PoseLandmark.LEFT_EYE].x * w - r_shldr_x
        left_eye_y = lm.landmark[mp_pose.PoseLandmark.LEFT_EYE].y * h - r_shldr_y
        left_eye_z = lm.landmark[mp_pose.PoseLandmark.LEFT_EYE].z * h
        right_eye_x = lm.landmark[mp_pose.PoseLandmark.RIGHT_EYE].x * w - r_shldr_x
        right_eye_y = lm.landmark[mp_pose.PoseLandmark.RIGHT_EYE].y * h - r_shldr_y
        right_eye_z = lm.landmark[mp_pose.PoseLandmark.RIGHT_EYE].z * h
        left_lip_x = lm.landmark[mp_pose.PoseLandmark.MOUTH_LEFT].x * w - r_shldr_x
        left_lip_y = lm.landmark[mp_pose.PoseLandmark.MOUTH_LEFT].y * h - r_shldr_y
        left_lip_z = lm.landmark[mp_pose.PoseLandmark.MOUTH_LEFT].z * h
        right_lip_x = lm.landmark[mp_pose.PoseLandmark.MOUTH_RIGHT].x * w - r_shldr_x
        right_lip_y = lm.landmark[mp_pose.PoseLandmark.MOUTH_RIGHT].y * h - r_shldr_y
        right_lip_z = lm.landmark[mp_pose.PoseLandmark.MOUTH_RIGHT].z * h
        r_shldr_x = 0
        r_shldr_y = 0

        writer.writerow([nose_x, nose_y, nose_z, l_shldr_x,l_shldr_y,l_shldr_z,r_shldr_x,r_shldr_y,r_shldr_z,left_eye_x, left_eye_y,left_eye_z, right_eye_x, right_eye_y,right_eye_z ,left_lip_x, left_lip_y, left_lip_z ,right_lip_x, right_lip_y,right_lip_z,label])

        # Write augmented image posture data to CSV file
        for augmented_image in augmented_images:
            augmented_keypoints = pose.process(augmented_image)
            augmented_lm = augmented_keypoints.pose_landmarks

            if augmented_lm is not None:
                nose_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
                nose_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
                nose_z_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

                l_shldr_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
                l_shldr_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
                l_shldr_z_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
                r_shldr_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
                r_shldr_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
                r_shldr_z_aug=augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
                l_shldr_x_aug -= r_shldr_x_aug
                l_shldr_y_aug -= r_shldr_y_aug
                nose_x_aug -=r_shldr_x_aug
                nose_y_aug -= r_shldr_y_aug

                left_eye_x_aug=augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_EYE].x * w- r_shldr_x_aug
                left_eye_y_aug=augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_EYE].y * h- r_shldr_y_aug
                left_eye_z_aug=augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_EYE].z * h
                right_eye_x_aug=augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_EYE].x * w- r_shldr_x_aug
                right_eye_y_aug=augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_EYE].y * h- r_shldr_y_aug
                right_eye_z_aug=augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_EYE].z * h
                left_lip_x_aug=augmented_lm.landmark[mp_pose.PoseLandmark.MOUTH_LEFT].x * w- r_shldr_x_aug
                left_lip_y_aug=augmented_lm.landmark[mp_pose.PoseLandmark.MOUTH_LEFT].y * h- r_shldr_y_aug
                left_lip_z_aug=augmented_lm.landmark[mp_pose.PoseLandmark.MOUTH_LEFT].z * h
                right_lip_x_aug=augmented_lm.landmark[mp_pose.PoseLandmark.MOUTH_RIGHT].x * w- r_shldr_x_aug
                right_lip_y_aug=augmented_lm.landmark[mp_pose.PoseLandmark.MOUTH_RIGHT].y * h- r_shldr_y_aug
                right_lip_z_aug=augmented_lm.landmark[mp_pose.PoseLandmark.MOUTH_RIGHT].z * h

                r_shldr_x_aug = 0
                r_shldr_y_aug = 0

                writer.writerow([nose_x_aug, nose_y_aug, nose_z_aug, l_shldr_x_aug,l_shldr_y_aug,l_shldr_z_aug,r_shldr_x_aug,r_shldr_y_aug,r_shldr_z_aug,left_eye_x,left_eye_y,left_eye_z,right_eye_x,right_eye_y,right_eye_z,left_lip_x,left_lip_y,left_lip_z,right_lip_x,right_lip_y,right_lip_z,label])
            else:
                print(f"No landmarks detected in augmented image {image_path}")

    else:
        print(f"No landmarks detected in image {image_path}")

def process_images_recursively(folder_path, writer, pose, datagen):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.lower().endswith(".jpg") or filename.lower().endswith(".png"):
                image_path = os.path.join(root, filename)
                print("Processing:", image_path)
                process_image(image_path, writer, pose, datagen)

def main():
    folder_path = '/content/drive/MyDrive/data/training/bad'
    csv_file_path = '/content/finbadtrb.csv'

    # Define data generator with augmentation parameters
    datagen = ImageDataGenerator(
        zoom_range=0.2,
        fill_mode='nearest')

    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Nose X', 'Nose Y', 'Nose Z','Left Shoulder X', 'Left Shoulder Y','Left Shoulder Z','Right Shoulder X' ,'Right Shoulder Y', 'Right Shoulder Z','Left Eye X','Left Eye Y','Left Eye Z','Right Eye X','Right Eye Y','Right Eye Z','Left Lip X','Left Lip Y','Left Lip Z','Right Lip X','Right Lip Y','Right Lip Z','Label'])

        pose = mp_pose.Pose()
        process_images_recursively(folder_path, writer, pose, datagen)
        pose.close()

    print('Finished processing images and saving data to CSV.')

if __name__ == "__main__":
    main()


Processing: /content/drive/MyDrive/data/training/bad/bad_posture_001.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_002.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_003.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_004.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_005.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_006.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_007.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_008.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_009.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_010.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_011.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_012.jpg
Processing: /content/drive/MyDrive/data/training/bad/bad_posture_013.jpg
Processing: /content/drive/MyDrive/data/training/ba

In [ ]:
import os
import cv2
import math
import mediapipe as mp
import csv
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

mp_pose = mp.solutions.pose

def findDistance(x1, y1, x2, y2):
    dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return dist

def findAngle(x1, y1, x2, y2):
    if (x2 - x1) == 0 or y1 == 0:
        return 0
    theta = math.acos((y2 - y1) * (-y1) / (math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) * y1))
    degree = int(180 / math.pi) * theta
    return degree

def process_image(image_path, writer, pose, datagen):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to read image from {image_path}")
        return

    h, w = image.shape[:2]
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Augment images
    augmented_images = []
    augmented_labels = []
    augmented_data = datagen.flow(np.expand_dims(image_rgb, axis=0), batch_size=1)
    for i in range(5):  # Augment each image 5 times
        augmented_image = augmented_data.next()[0].astype(np.uint8)
        augmented_images.append(augmented_image)
        augmented_labels.append('bad')

    keypoints = pose.process(image_rgb)
    lm = keypoints.pose_landmarks

    if lm is not None:
        nose_x = lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
        nose_y = lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
        nose_z = lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

        l_shldr_x = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
        l_shldr_y = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
        l_shldr_z = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
        r_shldr_x = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
        r_shldr_y = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
        r_shldr_z=lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
        l_shldr_x -= r_shldr_x
        l_shldr_y -= r_shldr_y
        nose_x -= r_shldr_x
        nose_y -= r_shldr_y

        label = 'bad'
        left_eye_x = lm.landmark[mp_pose.PoseLandmark.LEFT_EYE].x * w - r_shldr_x
        left_eye_y = lm.landmark[mp_pose.PoseLandmark.LEFT_EYE].y * h - r_shldr_y
        left_eye_z = lm.landmark[mp_pose.PoseLandmark.LEFT_EYE].z * h
        right_eye_x = lm.landmark[mp_pose.PoseLandmark.RIGHT_EYE].x * w - r_shldr_x
        right_eye_y = lm.landmark[mp_pose.PoseLandmark.RIGHT_EYE].y * h - r_shldr_y
        right_eye_z = lm.landmark[mp_pose.PoseLandmark.RIGHT_EYE].z * h
        left_lip_x = lm.landmark[mp_pose.PoseLandmark.MOUTH_LEFT].x * w - r_shldr_x
        left_lip_y = lm.landmark[mp_pose.PoseLandmark.MOUTH_LEFT].y * h - r_shldr_y
        left_lip_z = lm.landmark[mp_pose.PoseLandmark.MOUTH_LEFT].z * h
        right_lip_x = lm.landmark[mp_pose.PoseLandmark.MOUTH_RIGHT].x * w - r_shldr_x
        right_lip_y = lm.landmark[mp_pose.PoseLandmark.MOUTH_RIGHT].y * h - r_shldr_y
        right_lip_z = lm.landmark[mp_pose.PoseLandmark.MOUTH_RIGHT].z * h
        r_shldr_x = 0
        r_shldr_y = 0

        writer.writerow([nose_x, nose_y, nose_z, l_shldr_x,l_shldr_y,l_shldr_z,r_shldr_x,r_shldr_y,r_shldr_z,left_eye_x, left_eye_y,left_eye_z, right_eye_x, right_eye_y,right_eye_z ,left_lip_x, left_lip_y, left_lip_z ,right_lip_x, right_lip_y,right_lip_z,label])

        # Write augmented image posture data to CSV file
        for augmented_image in augmented_images:
            augmented_keypoints = pose.process(augmented_image)
            augmented_lm = augmented_keypoints.pose_landmarks

            if augmented_lm is not None:
                nose_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
                nose_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
                nose_z_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

                l_shldr_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
                l_shldr_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
                l_shldr_z_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
                r_shldr_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
                r_shldr_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
                r_shldr_z_aug=augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
                l_shldr_x_aug -= r_shldr_x_aug
                l_shldr_y_aug -= r_shldr_y_aug
                nose_x_aug -=r_shldr_x_aug
                nose_y_aug -= r_shldr_y_aug

                left_eye_x_aug=augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_EYE].x * w- r_shldr_x_aug
                left_eye_y_aug=augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_EYE].y * h- r_shldr_y_aug
                left_eye_z_aug=augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_EYE].z * h
                right_eye_x_aug=augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_EYE].x * w- r_shldr_x_aug
                right_eye_y_aug=augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_EYE].y * h- r_shldr_y_aug
                right_eye_z_aug=augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_EYE].z * h
                left_lip_x_aug=augmented_lm.landmark[mp_pose.PoseLandmark.MOUTH_LEFT].x * w- r_shldr_x_aug
                left_lip_y_aug=augmented_lm.landmark[mp_pose.PoseLandmark.MOUTH_LEFT].y * h- r_shldr_y_aug
                left_lip_z_aug=augmented_lm.landmark[mp_pose.PoseLandmark.MOUTH_LEFT].z * h
                right_lip_x_aug=augmented_lm.landmark[mp_pose.PoseLandmark.MOUTH_RIGHT].x * w- r_shldr_x_aug
                right_lip_y_aug=augmented_lm.landmark[mp_pose.PoseLandmark.MOUTH_RIGHT].y * h- r_shldr_y_aug
                right_lip_z_aug=augmented_lm.landmark[mp_pose.PoseLandmark.MOUTH_RIGHT].z * h

                r_shldr_x_aug = 0
                r_shldr_y_aug = 0

                writer.writerow([nose_x_aug, nose_y_aug, nose_z_aug, l_shldr_x_aug,l_shldr_y_aug,l_shldr_z_aug,r_shldr_x_aug,r_shldr_y_aug,r_shldr_z_aug,left_eye_x,left_eye_y,left_eye_z,right_eye_x,right_eye_y,right_eye_z,left_lip_x,left_lip_y,left_lip_z,right_lip_x,right_lip_y,right_lip_z,label])
            else:
                print(f"No landmarks detected in augmented image {image_path}")

    else:
        print(f"No landmarks detected in image {image_path}")

def process_images_recursively(folder_path, writer, pose, datagen):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.lower().endswith(".jpg") or filename.lower().endswith(".png"):
                image_path = os.path.join(root, filename)
                print("Processing:", image_path)
                process_image(image_path, writer, pose, datagen)

def main():
    folder_path = '/content/drive/MyDrive/data/validation/good'
    csv_file_path = '/content/finbadvalg.csv'

    # Define data generator with augmentation parameters
    datagen = ImageDataGenerator(
        zoom_range=0.2,
        fill_mode='nearest')

    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Nose X', 'Nose Y', 'Nose Z','Left Shoulder X', 'Left Shoulder Y','Left Shoulder Z','Right Shoulder X' ,'Right Shoulder Y', 'Right Shoulder Z','Left Eye X','Left Eye Y','Left Eye Z','Right Eye X','Right Eye Y','Right Eye Z','Left Lip X','Left Lip Y','Left Lip Z','Right Lip X','Right Lip Y','Right Lip Z','Label'])

        pose = mp_pose.Pose()
        process_images_recursively(folder_path, writer, pose, datagen)
        pose.close()

    print('Finished processing images and saving data to CSV.')

if __name__ == "__main__":
    main()


Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_001.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_002.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_003.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_004.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_005.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_006.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_007.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_19_12_21_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_19_24_44_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_19_24_46_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_19_24_47_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_19_24_56_Pro.jpg
Pro

In [ ]:
import os
import cv2
import math
import mediapipe as mp
import csv
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

mp_pose = mp.solutions.pose

def findDistance(x1, y1, x2, y2):
    dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return dist

def findAngle(x1, y1, x2, y2):
    if (x2 - x1) == 0 or y1 == 0:
        return 0
    theta = math.acos((y2 - y1) * (-y1) / (math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) * y1))
    degree = int(180 / math.pi) * theta
    return degree

def process_image(image_path, writer, pose, datagen):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to read image from {image_path}")
        return

    h, w = image.shape[:2]
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Augment images
    augmented_images = []
    augmented_labels = []
    augmented_data = datagen.flow(np.expand_dims(image_rgb, axis=0), batch_size=1)
    for i in range(5):  # Augment each image 5 times
        augmented_image = augmented_data.next()[0].astype(np.uint8)
        augmented_images.append(augmented_image)
        augmented_labels.append('bad')

    keypoints = pose.process(image_rgb)
    lm = keypoints.pose_landmarks

    if lm is not None:
        nose_x = lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
        nose_y = lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
        nose_z = lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

        l_shldr_x = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
        l_shldr_y = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
        l_shldr_z = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
        r_shldr_x = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
        r_shldr_y = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
        r_shldr_z=lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
        l_shldr_x -= r_shldr_x
        l_shldr_y -= r_shldr_y
        nose_x -= r_shldr_x
        nose_y -= r_shldr_y

        label = 'good'
        left_eye_x = lm.landmark[mp_pose.PoseLandmark.LEFT_EYE].x * w - r_shldr_x
        left_eye_y = lm.landmark[mp_pose.PoseLandmark.LEFT_EYE].y * h - r_shldr_y
        left_eye_z = lm.landmark[mp_pose.PoseLandmark.LEFT_EYE].z * h
        right_eye_x = lm.landmark[mp_pose.PoseLandmark.RIGHT_EYE].x * w - r_shldr_x
        right_eye_y = lm.landmark[mp_pose.PoseLandmark.RIGHT_EYE].y * h - r_shldr_y
        right_eye_z = lm.landmark[mp_pose.PoseLandmark.RIGHT_EYE].z * h
        left_lip_x = lm.landmark[mp_pose.PoseLandmark.MOUTH_LEFT].x * w - r_shldr_x
        left_lip_y = lm.landmark[mp_pose.PoseLandmark.MOUTH_LEFT].y * h - r_shldr_y
        left_lip_z = lm.landmark[mp_pose.PoseLandmark.MOUTH_LEFT].z * h
        right_lip_x = lm.landmark[mp_pose.PoseLandmark.MOUTH_RIGHT].x * w - r_shldr_x
        right_lip_y = lm.landmark[mp_pose.PoseLandmark.MOUTH_RIGHT].y * h - r_shldr_y
        right_lip_z = lm.landmark[mp_pose.PoseLandmark.MOUTH_RIGHT].z * h
        r_shldr_x = 0
        r_shldr_y = 0

        writer.writerow([nose_x, nose_y, nose_z, l_shldr_x,l_shldr_y,l_shldr_z,r_shldr_x,r_shldr_y,r_shldr_z,left_eye_x, left_eye_y,left_eye_z, right_eye_x, right_eye_y,right_eye_z ,left_lip_x, left_lip_y, left_lip_z ,right_lip_x, right_lip_y,right_lip_z,label])

        # Write augmented image posture data to CSV file
        for augmented_image in augmented_images:
            augmented_keypoints = pose.process(augmented_image)
            augmented_lm = augmented_keypoints.pose_landmarks

            if augmented_lm is not None:
                nose_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
                nose_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
                nose_z_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

                l_shldr_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
                l_shldr_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
                l_shldr_z_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
                r_shldr_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
                r_shldr_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
                r_shldr_z_aug=augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
                l_shldr_x_aug -= r_shldr_x_aug
                l_shldr_y_aug -= r_shldr_y_aug
                nose_x_aug -=r_shldr_x_aug
                nose_y_aug -= r_shldr_y_aug

                left_eye_x_aug=augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_EYE].x * w- r_shldr_x_aug
                left_eye_y_aug=augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_EYE].y * h- r_shldr_y_aug
                left_eye_z_aug=augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_EYE].z * h
                right_eye_x_aug=augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_EYE].x * w- r_shldr_x_aug
                right_eye_y_aug=augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_EYE].y * h- r_shldr_y_aug
                right_eye_z_aug=augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_EYE].z * h
                left_lip_x_aug=augmented_lm.landmark[mp_pose.PoseLandmark.MOUTH_LEFT].x * w- r_shldr_x_aug
                left_lip_y_aug=augmented_lm.landmark[mp_pose.PoseLandmark.MOUTH_LEFT].y * h- r_shldr_y_aug
                left_lip_z_aug=augmented_lm.landmark[mp_pose.PoseLandmark.MOUTH_LEFT].z * h
                right_lip_x_aug=augmented_lm.landmark[mp_pose.PoseLandmark.MOUTH_RIGHT].x * w- r_shldr_x_aug
                right_lip_y_aug=augmented_lm.landmark[mp_pose.PoseLandmark.MOUTH_RIGHT].y * h- r_shldr_y_aug
                right_lip_z_aug=augmented_lm.landmark[mp_pose.PoseLandmark.MOUTH_RIGHT].z * h

                r_shldr_x_aug = 0
                r_shldr_y_aug = 0

                writer.writerow([nose_x_aug, nose_y_aug, nose_z_aug, l_shldr_x_aug,l_shldr_y_aug,l_shldr_z_aug,r_shldr_x_aug,r_shldr_y_aug,r_shldr_z_aug,left_eye_x,left_eye_y,left_eye_z,right_eye_x,right_eye_y,right_eye_z,left_lip_x,left_lip_y,left_lip_z,right_lip_x,right_lip_y,right_lip_z,label])
            else:
                print(f"No landmarks detected in augmented image {image_path}")

    else:
        print(f"No landmarks detected in image {image_path}")

def process_images_recursively(folder_path, writer, pose, datagen):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.lower().endswith(".jpg") or filename.lower().endswith(".png"):
                image_path = os.path.join(root, filename)
                print("Processing:", image_path)
                process_image(image_path, writer, pose, datagen)

def main():
    folder_path = '/content/drive/MyDrive/data/validation/good'
    csv_file_path = '/content/finbadvalg.csv'

    # Define data generator with augmentation parameters
    datagen = ImageDataGenerator(
        zoom_range=0.2,
        fill_mode='nearest')

    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Nose X', 'Nose Y', 'Nose Z','Left Shoulder X', 'Left Shoulder Y','Left Shoulder Z','Right Shoulder X' ,'Right Shoulder Y', 'Right Shoulder Z','Left Eye X','Left Eye Y','Left Eye Z','Right Eye X','Right Eye Y','Right Eye Z','Left Lip X','Left Lip Y','Left Lip Z','Right Lip X','Right Lip Y','Right Lip Z','Label'])

        pose = mp_pose.Pose()
        process_images_recursively(folder_path, writer, pose, datagen)
        pose.close()

    print('Finished processing images and saving data to CSV.')

if __name__ == "__main__":
    main()


In [ ]:
import os
import cv2
import math
import mediapipe as mp
import csv
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

mp_pose = mp.solutions.pose

"""def findDistance(x1, y1, x2, y2):
    dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return dist

def findAngle(x1, y1, x2, y2):
    if (x2 - x1) == 0 or y1 == 0:
        return 0
    theta = math.acos((y2 - y1) * (-y1) / (math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) * y1))
    degree = int(180 / math.pi) * theta
    return degree"""

def process_image(image_path, writer, pose, datagen):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to read image from {image_path}")
        return

    h, w = image.shape[:2]
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Augment images
    augmented_images = []
    augmented_labels = []
    augmented_data = datagen.flow(np.expand_dims(image_rgb, axis=0), batch_size=1)
    for i in range(5):  # Augment each image 5 times
        augmented_image = augmented_data.next()[0].astype(np.uint8)
        augmented_images.append(augmented_image)
        augmented_labels.append('good')

    keypoints = pose.process(image_rgb)
    lm = keypoints.pose_landmarks

    if lm is not None:
        nose_x = lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
        nose_y = lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
        nose_z = lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

        l_shldr_x = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
        l_shldr_y = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
        l_shldr_z = lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
        r_shldr_x = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
        r_shldr_y = lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
        r_shldr_z=lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
        l_shldr_x -= r_shldr_x
        l_shldr_y -= r_shldr_y
        nose_x -= r_shldr_x
        nose_y -= r_shldr_y

        label = 'good'
        left_eye_x = lm.landmark[mp_pose.PoseLandmark.LEFT_EYE].x * w - r_shldr_x
        left_eye_y = lm.landmark[mp_pose.PoseLandmark.LEFT_EYE].y * h - r_shldr_y
        left_eye_z = lm.landmark[mp_pose.PoseLandmark.LEFT_EYE].z * h
        right_eye_x = lm.landmark[mp_pose.PoseLandmark.RIGHT_EYE].x * w - r_shldr_x
        right_eye_y = lm.landmark[mp_pose.PoseLandmark.RIGHT_EYE].y * h - r_shldr_y
        right_eye_z = lm.landmark[mp_pose.PoseLandmark.RIGHT_EYE].z * h
        left_lip_x = lm.landmark[mp_pose.PoseLandmark.MOUTH_LEFT].x * w - r_shldr_x
        left_lip_y = lm.landmark[mp_pose.PoseLandmark.MOUTH_LEFT].y * h - r_shldr_y
        left_lip_z = lm.landmark[mp_pose.PoseLandmark.MOUTH_LEFT].z * h
        right_lip_x = lm.landmark[mp_pose.PoseLandmark.MOUTH_RIGHT].x * w - r_shldr_x
        right_lip_y = lm.landmark[mp_pose.PoseLandmark.MOUTH_RIGHT].y * h - r_shldr_y
        right_lip_z = lm.landmark[mp_pose.PoseLandmark.MOUTH_RIGHT].z * h
        r_shldr_x = 0
        r_shldr_y = 0

        writer.writerow([nose_x, nose_y, nose_z, l_shldr_x,l_shldr_y,l_shldr_z,r_shldr_x,r_shldr_y,r_shldr_z,left_eye_x, left_eye_y,left_eye_z, right_eye_x, right_eye_y,right_eye_z ,left_lip_x, left_lip_y, left_lip_z ,right_lip_x, right_lip_y,right_lip_z,label])

        # Write augmented image posture data to CSV file
        for augmented_image in augmented_images:
            augmented_keypoints = pose.process(augmented_image)
            augmented_lm = augmented_keypoints.pose_landmarks

            if augmented_lm is not None:
                nose_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].x * w
                nose_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].y * h
                nose_z_aug = augmented_lm.landmark[mp_pose.PoseLandmark.NOSE].z * h  # Calculating Z-axis position

                l_shldr_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w
                l_shldr_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h
                l_shldr_z_aug = augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].z * h
                r_shldr_x_aug = augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w
                r_shldr_y_aug = augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h
                r_shldr_z_aug=augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].z * h
                l_shldr_x_aug -= r_shldr_x_aug
                l_shldr_y_aug -= r_shldr_y_aug
                nose_x_aug -=r_shldr_x_aug
                nose_y_aug -= r_shldr_y_aug

                left_eye_x_aug=augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_EYE].x * w- r_shldr_x_aug
                left_eye_y_aug=augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_EYE].y * h- r_shldr_y_aug
                left_eye_z_aug=augmented_lm.landmark[mp_pose.PoseLandmark.LEFT_EYE].z * h
                right_eye_x_aug=augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_EYE].x * w- r_shldr_x_aug
                right_eye_y_aug=augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_EYE].y * h- r_shldr_y_aug
                right_eye_z_aug=augmented_lm.landmark[mp_pose.PoseLandmark.RIGHT_EYE].z * h
                left_lip_x_aug=augmented_lm.landmark[mp_pose.PoseLandmark.MOUTH_LEFT].x * w- r_shldr_x_aug
                left_lip_y_aug=augmented_lm.landmark[mp_pose.PoseLandmark.MOUTH_LEFT].y * h- r_shldr_y_aug
                left_lip_z_aug=augmented_lm.landmark[mp_pose.PoseLandmark.MOUTH_LEFT].z * h
                right_lip_x_aug=augmented_lm.landmark[mp_pose.PoseLandmark.MOUTH_RIGHT].x * w- r_shldr_x_aug
                right_lip_y_aug=augmented_lm.landmark[mp_pose.PoseLandmark.MOUTH_RIGHT].y * h- r_shldr_y_aug
                right_lip_z_aug=augmented_lm.landmark[mp_pose.PoseLandmark.MOUTH_RIGHT].z * h

                r_shldr_x_aug = 0
                r_shldr_y_aug = 0

                writer.writerow([nose_x_aug, nose_y_aug, nose_z_aug, l_shldr_x_aug,l_shldr_y_aug,l_shldr_z_aug,r_shldr_x_aug,r_shldr_y_aug,r_shldr_z_aug,left_eye_x,left_eye_y,left_eye_z,right_eye_x,right_eye_y,right_eye_z,left_lip_x,left_lip_y,left_lip_z,right_lip_x,right_lip_y,right_lip_z,label])
            else:
                print(f"No landmarks detected in augmented image {image_path}")

    else:
        print(f"No landmarks detected in image {image_path}")

def process_images_recursively(folder_path, writer, pose, datagen):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.lower().endswith(".jpg") or filename.lower().endswith(".png"):
                image_path = os.path.join(root, filename)
                print("Processing:", image_path)
                process_image(image_path, writer, pose, datagen)

def main():
    folder_path = '/content/drive/MyDrive/data/validation/good'
    csv_file_path = '/content/fingoodvalg.csv'

    # Define data generator with augmentation parameters
    datagen = ImageDataGenerator(
        zoom_range=0.2,
        fill_mode='nearest')

    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Nose X', 'Nose Y', 'Nose Z','Left Shoulder X', 'Left Shoulder Y','Left Shoulder Z','Right Shoulder X' ,'Right Shoulder Y', 'Right Shoulder Z','Left Eye X','Left Eye Y','Left Eye Z','Right Eye X','Right Eye Y','Right Eye Z','Left Lip X','Left Lip Y','Left Lip Z','Right Lip X','Right Lip Y','Right Lip Z','Label'])

        pose = mp_pose.Pose()
        process_images_recursively(folder_path, writer, pose, datagen)
        pose.close()

    print('Finished processing images and saving data to CSV.')

if __name__ == "__main__":
    main()


Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_001.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_002.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_003.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_004.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_005.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_006.jpg
Processing: /content/drive/MyDrive/data/validation/good/good_posture_val_007.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_19_12_21_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_19_24_44_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_19_24_46_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_19_24_47_Pro.jpg
Processing: /content/drive/MyDrive/data/validation/good/WIN_20240429_19_24_56_Pro.jpg
Pro